## Section 1: Import Libraries and set runtime variables

This script sets up an environment for data analysis and visualization.

## Imports

- **Standard Libraries**: `os`, `warnings`
- **Data Analysis and Visualization**: `matplotlib.pyplot`, `numpy`, `pandas`, `seaborn`, `calendar`, `matplotlib.dates`
- **IPython Specific**: `IPython.core.magic`, `aquarel`

## Configuration

- **Warnings**: Ignored to keep output clean.
- **IPython Instance**: Enables dynamic execution of code cells.

## Custom Cell Magic

- **`skip_if` Magic Command**:
  - Skips cell execution based on a condition.
  - Useful for skipping data cleaning steps in reruns once the dataframe is cleaned.

## Variables

- **`no_cleaning`**: Ensures data cleaning steps are not skipped during reruns.
- **`all_resources_csv_available`**: Indicates availability of necessary resource files.


In [ ]:
# Import necessary libraries
import os
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import calendar
import matplotlib.dates as mat_dates
import re

# Import specific functions from libraries
from IPython.core.magic import register_cell_magic
from aquarel import load_theme

# Ignore warnings to keep the output clean
warnings.filterwarnings('ignore')

# Get the IPython instance to interact with the IPython environment
ipython = get_ipython()

# Define a custom cell magic command to skip data cleaning steps in subsequent runs
@register_cell_magic
def skip_if(line, cell):
    # Evaluate the condition provided in the line argument
    if eval(line):
        return  # Skip the cell if the condition is True
    # Otherwise, execute the cell
    ipython.run_cell(cell)

# If this variable is set to True. Skips cleaning during reruns of code if not required
no_cleaning = False

# Set this variable to True if all Resources Excel files downloaded from NHS are available
# For submission, this variable is False as only the cleaned up combined CSV is submitted
all_resources_csv_available = False


## Section 2: Clean up Actual Duration Dataset

### Loading Data

- **Pickle or CSV**: The script checks for the existence of a pickle file (`actual_duration.pickle`). If it exists, it loads the data from the pickle file; otherwise, it reads the CSV file (`actual_duration.csv`) and converts it to a pickle file for faster loading in subsequent runs.
- Sense check the data set using different functions.

### Data Cleaning

- **Handling Missing Values**: Identifies and drops rows with missing values.
- **Removing Duplicates**: Identifies and drops duplicate rows, keeping only the first occurrence.
- **Resetting Index**: Resets the index of the DataFrame after dropping rows.

### Data Transformation

- **Splitting Columns**: Splits `sub_icb_location_name` into `icb_location_name` and `sub_icb_location_code`.
- **Date Handling**: Converts `appointment_date` to datetime, extracts month-year, and determines the day of the week.
- **Categorical Data**: Converts specified columns to categorical data types and defines the order for days of the week and appointment durations.

### Reordering Columns

- **Column Reordering**: Reorders the columns in the DataFrame for better readability and structure.


In [ ]:
%%skip_if no_cleaning
# This cell will be skipped if no_cleaning is True

# If pickle file exists, read it; otherwise, read the CSV and convert to pickle
# This ensures faster loading from the second run and avoids reading the CSV file when resetting the kernel
if not os.path.exists("actual_duration.pickle"):
    # Read the CSV file and convert it to a pickle file
    ad = pd.read_csv('actual_duration.csv')
    ad.to_pickle('actual_duration.pickle')
    # Read the newly created pickle file
    ad = pd.read_pickle('actual_duration.pickle')
else:
    # Read the existing pickle file
    ad = pd.read_pickle('actual_duration.pickle')

# View the DataFrame (commented out for execution)
# print("Head \n", ad.head())
# print("Types\n", ad.dtypes)
# print("Shape\n", ad.shape)
# print("Describe \n", ad.describe)


In [ ]:
%%skip_if no_cleaning
# This cell will be skipped if no_cleaning is True

# Data Cleaning

# Determine whether there are missing values in the DataFrame
ad_na = ad[ad.isna().any(axis=1)]

# Drop rows with missing values from the DataFrame
ad.drop(ad_na.index, axis=0, inplace=True)

# Identify duplicate rows in the DataFrame
ad_duplicates = ad[ad.duplicated(keep=False)]

# Drop duplicate rows from the DataFrame, keeping the first occurrence
ad.drop_duplicates(keep='first', inplace=True)

# Reset the index of the DataFrame after dropping rows
ad.reset_index(drop=True, inplace=True)

In [ ]:
%%skip_if no_cleaning
# This cell will be skipped if no_cleaning is True
# Split 'sub_icb_location_name' into 'icb_location_name' and 'sub_icb_location_code'
# for consistency with the National Categories database
ad[['icb_location_name', 'sub_icb_location_code']] = ad['sub_icb_location_name'].str.rsplit(' - ', expand=True)

# Change Date to datetime and add day of the week
ad['appointment_date'] = pd.to_datetime(ad['appointment_date'])
ad['appointment_month_year'] = ad['appointment_date'].dt.strftime('%Y-%m')
ad['appointment_month_year'] = pd.to_datetime(ad['appointment_month_year'])
ad['appointment_day_of_the_week'] = ad['appointment_date'].dt.day_name()

# Convert specified columns to categorical data type
ad['sub_icb_location_code'] = ad['sub_icb_location_code'].astype('category')
ad['sub_icb_location_name'] = ad['sub_icb_location_name'].astype('category')
ad['sub_icb_location_ons_code'] = ad['sub_icb_location_ons_code'].astype('category')
ad['icb_location_name'] = ad['icb_location_name'].astype('category')
ad['icb_ons_code'] = ad['icb_ons_code'].astype('category')
ad['region_ons_code'] = ad['region_ons_code'].astype('category')

# Define the order of the days of the week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Convert the 'appointment_day_of_the_week' column to a categorical type with the specified order
ad['appointment_day_of_the_week'] = pd.Categorical(
    ad['appointment_day_of_the_week'],
    categories=day_order,
    ordered=True
)

#Change actual_duration to ordered categorical and add average of high and low as value, unknown/ data quality set as 999
duration_order = ['1-5 Minutes', '6-10 Minutes', '11-15 Minutes',
                  '16-20 Minutes', '21-30 Minutes', '31-60 Minutes',
                  'Unknown / Data Quality']

ad['actual_duration'] = pd.Categorical(
    ad['actual_duration'],
    categories=duration_order,
    ordered=True
)

In [ ]:
%%skip_if no_cleaning
# This cell will be skipped if no_cleaning is True

# Reorder the columns in the DataFrame for better readability and structure
ad = ad[['region_ons_code', 'icb_ons_code', 'icb_location_name', 'sub_icb_location_code',
         'appointment_month_year', 'appointment_day_of_the_week', 'actual_duration',
         'count_of_appointments','sub_icb_location_ons_code', 'sub_icb_location_name']]

## Section 3: Clean up National Categories Dataset

### Loading Data

- **Pickle or Excel**: The script checks for the existence of a pickle file (`national_categories.pickle`). If it exists, it loads the data from the pickle file; otherwise, it reads the Excel file (`national_categories.xlsx`) and converts it to a pickle file for faster loading in subsequent runs.
- Sense check the data set using different functions.

### Data Cleaning

- **Handling Missing Values**: Identifies and drops rows with missing values.
- **Removing Duplicates**: Identifies and drops duplicate rows, keeping only the first occurrence.
- **Resetting Index**: Resets the index of the DataFrame after dropping rows.

### Data Transformation

- **Splitting Columns**: Splits `sub_icb_location_name` into `icb_location_name` and `sub_icb_location_code`.
- **Date Handling**: Converts `appointment_date` to datetime, extracts month-year, and determines the day of the week.
- **Summarizing Locations**: Creates summaries of all locations and codes in the dataset.

### Categorical Data

- **Categorical Conversion**: Converts specified columns to categorical data types and defines the order for days of the week.

### Season Determination

- **Season Function**: Defines a function to determine the season based on the appointment year and month.
- **Apply Function**: Applies the function to create a new column `season_year`.

### Reordering Columns

- **Column Reordering**: Reorders the columns in the DataFrame for better readability and structure.


In [ ]:
%%skip_if no_cleaning
# This cell will be skipped if no_cleaning is True

# Check if the pickle file exists to read from it; otherwise, read from the Excel file
# This ensures faster loading from the second run and avoids reading the Excel file when resetting the kernel
if not os.path.exists("national_categories.pickle"):
    # Read the Excel file and convert it to a pickle file
    nc = pd.read_excel('national_categories.xlsx')
    nc.to_pickle('national_categories.pickle')
    # Read the newly created pickle file
    nc = pd.read_pickle('national_categories.pickle')
else:
    # Read the existing pickle file
    nc = pd.read_pickle('national_categories.pickle')

# View the DataFrame (commented out for execution)
# print("Head \n", nc.head())
# print("Types\n", nc.dtypes)
# print("Shape\n", nc.shape)
# print("Describe \n", nc.describe)


In [ ]:
%%skip_if no_cleaning
# This cell will be skipped if no_cleaning is True

# Data Cleaning

# Determine whether there are missing values in the DataFrame
nc_na = nc[nc.isna().any(axis=1)]
# ar_na.info() # No NA values found; dropping for consistency or update in dataset
nc.drop(nc_na.index, axis=0, inplace=True)

# Identify duplicate rows in the DataFrame
nc_duplicates = nc[nc.duplicated(keep=False)]
# ar_duplicates.info() # No duplicate values found; dropping for consistency or update in dataset
nc.drop_duplicates(keep='first', inplace=True)

# Reset the index of the DataFrame after dropping rows
nc.reset_index(drop=True, inplace=True)

In [ ]:
%%skip_if no_cleaning
# This cell will be skipped if no_cleaning is True

# Split sub_icb_location_name into ICB_location_name and sub_icb_location_code
nc[['icb_location_name', 'sub_icb_location_code']] = nc['sub_icb_location_name'].str.rsplit(' - ', expand=True)

# Change Date to datetime and add day of the week
nc['appointment_date'] = pd.to_datetime(nc['appointment_date'])
nc['appointment_month_year'] = nc['appointment_date'].dt.strftime('%Y-%m')
nc['appointment_month_year'] = pd.to_datetime(nc['appointment_month_year'])
nc['appointment_day_of_the_week'] = nc['appointment_date'].dt.day_name()

# Create summary of all locations and codes in the dataset
sub_icb_summary = nc[['sub_icb_location_code', 'icb_location_name', 'icb_ons_code']].drop_duplicates(
    keep='first').reset_index(drop=True)
icb_summary = sub_icb_summary[['icb_location_name', 'icb_ons_code']].drop_duplicates(keep='first').reset_index(
    drop=True)

In [ ]:
%%skip_if no_cleaning
# This cell will be skipped if no_cleaning is True

# Convert specified columns to categorical data type
nc['icb_ons_code'] = nc['icb_ons_code'].astype('category')
nc['icb_location_name'] = nc['icb_location_name'].astype('category')
nc['sub_icb_location_code'] = nc['sub_icb_location_code'].astype('category')
nc['service_setting'] = nc['service_setting'].astype('category')
nc['context_type'] = nc['context_type'].astype('category')
nc['national_category'] = nc['national_category'].astype('category')

nc['appointment_day_of_the_week'] = pd.Categorical(
    nc['appointment_day_of_the_week'],
    categories=day_order,
    ordered=True
)


In [ ]:
%%skip_if no_cleaning

# This cell will be skipped if no_cleaning is True

# Define a function to determine the season based on the appointment year and month
def get_season(row):
    """
    Determine the season based on the appointment year and month.

    This function categorizes a given date into a specific season based on the year and month.
    It is designed to handle dates within the years 2021 and 2022, returning a string that
    represents the season for the given date.

    Parameters:\n
    - row (pd.Series): A row from a DataFrame containing the following columns:\n
        - appointment_year (int): The year of the appointment.
        - appointment_month_num (int): The month of the appointment as a number (1-12).

    Returns:\n
    - str or None: A string representing the season for the given date, or None if the date
      does not fall within the specified seasons for 2021 and 2022.

    Season Definitions:\n
    - Summer 2021: June to August 2021
    - Autumn 2021: September to November 2021
    - Winter 2021 - 2022: December 2021 to February 2022
    - Spring 2022: March to May 2022

    """

    year = row['appointment_year']
    month = row['appointment_month_num']

    if (month >= 6) and (month <= 8) and (year == 2021):
        return 'Summer 2021'
    elif (month >= 9) and (month <= 11) and (year == 2021):
        return 'Autumn 2021'
    elif (month == 12 and year == 2021) or (month <= 2 and year == 2022):
        return 'Winter 2021 - 2022'
    elif (3 <= month <= 5) and (year == 2022):
        return 'Spring 2022'
    else:
        return None


# Apply the function to create a new column
nc['appointment_year'] = nc['appointment_date'].dt.year
nc['appointment_month_num'] = nc['appointment_date'].dt.month
nc['season_year'] = nc.apply(get_season, axis=1)


In [ ]:
%%skip_if no_cleaning
# This cell will be skipped if no_cleaning is True

# Reorder the columns in the DataFrame for better readability and structure
nc = nc[['icb_ons_code', 'icb_location_name', 'sub_icb_location_code', 'appointment_date',
               'appointment_month_year', 'appointment_day_of_the_week', 'service_setting',
               'context_type', 'national_category', 'count_of_appointments', 'sub_icb_location_name', 'season_year']]


## Section 4: Clean up Appointments Regional Dataset

## Loading Data

- **Pickle or CSV**: The script checks for the existence of a pickle file (`appointments_regional.pickle`). If it exists, it loads the data from the pickle file; otherwise, it reads the CSV file (`appointments_regional.csv`) and converts it to a pickle file for faster loading in subsequent runs.
- Sense check the data set using different functions.

## Data Cleaning

- **Handling Missing Values**: Identifies and drops rows with missing values.
- **Removing Duplicates**: Identifies and drops duplicate rows, keeping only the first occurrence.
- **Resetting Index**: Resets the index of the DataFrame after dropping rows.

## Data Transformation

- **Merging Data**: Adds ICB names to the table by merging with `icb_summary`.
- **Categorical Data**: Converts specified columns to categorical data types and defines the order for `time_between_book_and_appointment`.

## Date Handling

- **Date Conversion**: Converts `appointment_month` to datetime and formats `appointment_month_year`.

## Reordering Columns

- **Column Reordering**: Reorders the columns in the DataFrame for better readability and structure.



In [ ]:
%%skip_if no_cleaning
# This cell will be skipped if no_cleaning is True

# If pickle file exists, read pickle file; if not, read the CSV, convert to pickle, and read the pickle
# This ensures faster loading from the second run and avoids reading the CSV file when resetting the kernel
if not os.path.exists("appointments_regional.pickle"):
    ar = pd.read_csv('appointments_regional.csv')
    ar.to_pickle('appointments_regional.pickle')
    ar = pd.read_pickle('appointments_regional.pickle')
else:
    ar = pd.read_pickle('appointments_regional.pickle')

# View the DataFrame (commented out for execution)
# print("Head \n", ar.head())
# print("Types\n", ar.dtypes)
# print("Shape\n", ar.shape)
# print("Describe \n", ar.describe)

In [ ]:
%%skip_if no_cleaning
# This cell will be skipped if no_cleaning is True
# Data Cleaning

# Determine whether there are missing values in the DataFrame
ar_na = ar[ar.isna().any(axis=1)]
# ar_na.info() # No NA values found; dropping for consistency or update in dataset
ar.drop(ar_na.index, axis=0, inplace=True)

# Identify duplicate rows in the DataFrame
ar_duplicates = ar[ar.duplicated(keep=False)]
# ar_duplicates.info() # No duplicate values found; dropping for consistency or update in dataset
ar.drop_duplicates(keep='first', inplace=True)

# Reset the index of the DataFrame after dropping rows
ar.reset_index(drop=True, inplace=True)


In [ ]:
%%skip_if no_cleaning
# This cell will be skipped if no_cleaning is True

#Add ICB Names to table
ar = ar.merge(icb_summary[["icb_ons_code", 'icb_location_name']], on='icb_ons_code', how='left')

In [ ]:
%%skip_if no_cleaning
# This cell will be skipped if no_cleaning is True

# Order the category time between book and appointment order
time_between_order = ['Same Day', '1 Day', '2 to 7 Days', '8  to 14 Days',
                                           '15  to 21 Days', '22  to 28 Days', 'More than 28 Days',
                                           'Unknown / Data Quality']

# Convert 'time_between_book_and_appointment' to ordered categorical type
ar['time_between_book_and_appointment'] = pd.Categorical(
    ar['time_between_book_and_appointment'],
    categories=time_between_order,
    ordered=True
)

In [ ]:
%%skip_if no_cleaning
# This cell will be skipped if no_cleaning is True

# Convert specified columns to categorical data type
ar['icb_ons_code'] = ar['icb_ons_code'].astype('category')
ar['appointment_status'] = ar['appointment_status'].astype('category')
ar['hcp_type'] = ar['hcp_type'].astype('category')
ar['appointment_mode'] = ar['appointment_mode'].astype('category')
ar['time_between_book_and_appointment'] = ar['time_between_book_and_appointment'].astype('category')
ar['icb_location_name'] = ar['icb_location_name'].astype('category')

# Convert 'appointment_month' to datetime and format 'appointment_month_year'
ar['appointment_month'] = pd.to_datetime(ar['appointment_month'])
ar['appointment_month_year'] = ar['appointment_month'].dt.strftime('%Y-%m')
ar['appointment_month_year'] = pd.to_datetime(ar['appointment_month_year'])

In [ ]:
%%skip_if no_cleaning
# This cell will be skipped if no_cleaning is True

# Reorder the columns in the DataFrame for better readability and structure
ar = ar[['icb_ons_code', 'icb_location_name', 'appointment_month_year', 'appointment_mode',
               'hcp_type', 'appointment_status', 'time_between_book_and_appointment', 'count_of_appointments', ]]


## Section 5: Clean up GP Resources Dataset obtained from digital.nhs.uk

# GP Individual Level Data Combined

## Function: `clean_excel_resources_gp`

- **Purpose**: Cleans and processes GP (General Practitioner) resources data from a CSV file for a given year and month.
- **Parameters**:
  - `year` (int): The year of the data to be processed.
  - `month` (int): The month of the data to be processed.
- **Returns**:
  - A cleaned and processed DataFrame containing the following columns:
    - `appointment_month`: The month of the appointment as a datetime.
    - `sub_icb_location_code`: The sub ICB location code.
    - `staff_group`: The staff group.
    - `fte_total`: The total Full-Time Equivalent (FTE) count.
    - `employee_count`: The count of employees.

### Steps:

1. **Read the CSV File**: For the specified year and month.
2. **Drop Missing Values**: Remove rows with missing values.
3. **Drop Duplicates**: Remove duplicate rows, keeping the first occurrence.
4. **Combine Year and Month**: Into a datetime column.
5. **Filter and Process Data**:
   - For data from June 2022 onwards, use 'SUB_ICB_CODE'.
   - For data before June 2022, use 'CCG_CODE'.
6. **Drop Non-Provided Data**: Remove rows where data is not provided (i.e., estimated or estimated FTE).
7. **Group Data**: By month, location code, and staff group, summing FTE and counting employees.
8. **Rename Columns**: For consistency.
9. **Correct Values**: In the 'staff_group' column.
10. **Return DataFrame**: The cleaned and processed DataFrame.

## Data Loading

- **Pickle or CSV**: The script checks for the existence of a pickle file (`resources_data.pickle`). If it exists, it loads the data from the pickle file; otherwise, it reads the CSV files for the specified date range, cleans each file, combines them, and converts the result to a pickle file for faster loading in subsequent runs.

### Conditions:

- **All Resources CSV Available**: If all resources CSV files from NHS are available, it processes files from June 2021 to June 2022. (format of Date YYYY-MM IL.csv)
- **Otherwise**: It reads the combined CSV file (`resources_data.csv`) and converts it to a pickle file if it doesn't already exist.


In [ ]:
%%skip_if no_cleaning

# This cell will be skipped if no_cleaning is True

#Source - https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services?__cf_chl_tk=QxyUf1IvCHmwHJAN8fQtOuO76KntPb73rT.CRzdQ5as-1747576648-1.0.1.1-JMqb6Q7TZgD06dvBQsWHk2SNT_foXdD_sqPZlbL2Kz8#past-publications

# sub_icb_code is the same as sub_icb_location code. - https://www.sbs.nhs.uk/supplier-information/ccg-icb-list/

# GP Individual Level Data combined
def clean_excel_resources_gp(year, month):
    """
    Clean and process GP (General Practitioner) resources data from a CSV file for a given year and month.

    This function reads a CSV file containing GP resources data, performs data cleaning operations,
    and processes the data based on the specified year and month. It handles changes in column names
    that occurred from June 2022 and filters out non-provided data.

    Parameters:\n
    - year (int): The year of the data to be processed.
    - month (int): The month of the data to be processed.

    Returns:\n
    - pd.DataFrame: A cleaned and processed DataFrame containing the following columns:\n
        - appointment_month: The month of the appointment as a datetime.
        - sub_icb_location_code: The sub ICB location code.
        - staff_group: The staff group.
        - fte_total: The total Full-Time Equivalent (FTE) count.
        - employee_count: The count of employees.

    Steps:\n
    1. Read the CSV file for the specified year and month.
    2. Drop rows with missing values.
    3. Drop duplicate rows, keeping the first occurrence.
    4. Combine Year and Month into a datetime column.
    5. Filter and process data based on the year and month:
        - For data from June 2022 onwards, use 'SUB_ICB_CODE'.
        - For data before June 2022, use 'CCG_CODE'.
    6. Drop rows where data is not provided (i.e., estimated or estimated FTE).
    7. Group by month, location code, and staff group, summing FTE and counting employees.
    8. Rename columns for consistency.
    9. Return the cleaned and processed DataFrame.
    """

    rgp_function = pd.read_csv("Resources/" + str(year) + "-" + str(month).zfill(2) + " IL.csv")

    #Drop NA
    read_data_resources_na = rgp_function[rgp_function.isna().any(axis=1)]
    rgp_function.drop(read_data_resources_na.index, axis=0, inplace=True)

    #Drop Duplicates
    rgp_function.drop_duplicates(keep="first", inplace=True)

    # #Keep only necessary columns
    rgp_function['appointment_month_year'] = pd.to_datetime(
        rgp_function[['YEAR', 'Month']].assign(DAY=1))  #Combine Year/ Month into datetime

    #CCG Code change to SUB_ICB_CODE from June 2022
    if year >= 2022 and month >= 6:
        rgp_function = rgp_function[
            ['appointment_month_year', 'SUB_ICB_CODE', 'DATA_SOURCE', 'STAFF_GROUP', 'FTE']]  #Keep only specified columns

        # #Drop if data is not provided ie estimated or estimated FTE
        index_names = rgp_function[(rgp_function['DATA_SOURCE'] != 'Provided')].index
        rgp_function.drop(index_names, inplace=True)

        #Group by Month, SUB ICB Code, Staff Group and sum FTE and also count
        rgp_function = rgp_function.groupby(['appointment_month_year', 'SUB_ICB_CODE', 'STAFF_GROUP']).agg(
            FTETotal=('FTE', 'sum'), EmployeeCount=('FTE', 'count')).reset_index()

        rgp_function.rename(columns={'SUB_ICB_CODE': 'sub_icb_location_code'}, inplace=True)

    else:
        rgp_function = rgp_function[['appointment_month_year', 'CCG_CODE', 'DATA_SOURCE', 'STAFF_GROUP', 'FTE']]

        # #Drop if data is not provided ie estimated or estimated FTE
        index_names = rgp_function[(rgp_function['DATA_SOURCE'] != 'Provided')].index
        rgp_function.drop(index_names, inplace=True)

        #Group by Month, CCG Code, Staff Group and sum FTE and also count
        rgp_function = rgp_function.groupby(['appointment_month_year', 'CCG_CODE', 'STAFF_GROUP']).agg(
            FTETotal=('FTE', 'sum'), EmployeeCount=('FTE', 'count')).reset_index()

        rgp_function.rename(columns={'CCG_CODE': 'sub_icb_location_code'}, inplace=True)

    #Rename CCG Code to SUB_ICB_CODE

    rgp_function.rename(columns={'STAFF_GROUP': 'staff_group'}, inplace=True)
    rgp_function.rename(columns={'EmployeeCount': 'employee_count'}, inplace=True)
    rgp_function.rename(columns={'FTETotal': 'fte_total'}, inplace=True)

     # Correct the values in the 'staff_group' column
    rgp_function['staff_group'] = rgp_function['staff_group'].str.replace('Admin/Non-clinical', 'Admin/Non-Clinical')

    return rgp_function


In [ ]:
%%skip_if no_cleaning
# This cell will be skipped if no_cleaning is True

# If pickle file exists, read pickle file; if not, read the CSV, convert to pickle, and read the pickle
# This ensures faster loading from the second run and avoids reading the CSV file when resetting the kernel
if all_resources_csv_available:
    if not os.path.exists("resources_data.pickle"):
        # Read files from June 2021 to June 2022, use function to clean each file, append them to a list, at the end join all dataframes

        itr_month = 6
        itr_year = 2021
        end_month = 6
        end_year = 2022
        rgp_list = []

        while itr_year < end_year or (itr_year == end_year and itr_month <= end_month):
            # Clean and append each file to the list
            rgp_list.append(clean_excel_resources_gp(itr_year, itr_month))

            # Increment year and reset month if necessary
            if itr_month == 12:
                itr_year += 1
                itr_month = 1
            else:
                itr_month += 1

        # Concatenate all dataframes in the list
        rgp = pd.concat(rgp_list, ignore_index=True)
        rgp.to_pickle('resources_data.pickle')
        rgp.to_csv('resources_data.csv')

    else:
        # Read the existing pickle file
        rgp = pd.read_pickle('resources_data.pickle')

else:
    if not os.path.exists("resources_data.pickle"):
        rgp = pd.read_csv('resources_data.csv')
        rgp.to_pickle('resources_data.pickle')
        rgp = pd.read_pickle('resources_data.pickle')
    else:
        rgp = pd.read_pickle('resources_data.pickle')

# View the DataFrame (commented out for execution)
# print("Head \n", rgp.head())
# print("Types\n", rgp.dtypes)
# print("Shape\n", rgp.shape)
# print("Describe \n", rgp.describe)


## Section 6: Import theme and create standardized formatting for graphs

# Visual Theme and Plotting Functions

## Visual Theme

- **Consistent Theme**: Applies a consistent visual theme throughout the project using the `boxy_dark` theme.
- **Standard Dimensions**: Defines standard dimensions for plots with a width of 15 and a height of 6.
- **Markers**: Defines a list of markers to be used.

## Plotting Functions

### Function: `plot_display`

- **Purpose**: Plots a graph with specified title, axis labels, and optional annotations.
- **Parameters**:
  - `title` (str): Title of the plot.
  - `xlabel` (str): Label for the x-axis.
  - `ylabel` (str): Label for the y-axis.
  - `legendlabel` (str, optional): Label for the legend. If None, no legend is displayed.
  - `data` (pd.DataFrame, optional): DataFrame containing the data to annotate for min/max. Defaults to None.
  - `ycolumn` (str, optional): Column name in data for y-axis values. Defaults to None.
  - `xcolumn` (str, optional): Column name in data for x-axis values. Defaults to None.
  - `huecolumn` (str, optional): Column name in data for hue categories. Defaults to None.
  - `xticksrotation` (int, optional): Rotation angle for x-axis tick labels. Defaults to 45.
  - `legendlocation` (str, optional): Location of the legend. Defaults to 'upper left'.
  - `legendanchor` (tuple, optional): Anchor point for the legend. Defaults to (1, 1).
  - `legendboxpadding` (int, optional): Padding around the legend box. Defaults to 1.
  - `ax` (matplotlib.axes.Axes, optional): Axes to plot on. If None, uses the current axes. Defaults to None.
  - `format_datetime` (bool, optional): Whether to format the x-axis as date-time. Defaults to False.

#### Steps:
1. **Set Plot Attributes**: Title, x-axis label, y-axis label, and x-axis tick rotation.
2. **Format X-Axis**: As date-time if specified.
3. **Annotate Min/Max Values**: If data is provided, annotate the minimum and maximum values on the plot.
4. **Add Legend**: If a legend label is provided.
5. **Show Plot**: If axes are not provided, show the plot with tight layout.

### Function: `annotate_min_max`

- **Purpose**: Annotates the minimum and maximum values on the plot for each category.
- **Parameters**:
  - `data` (pd.DataFrame): DataFrame containing the data.
  - `yaxisdata` (str): Column name in data for y-axis values.
  - `xasixdata` (str): Column name in data for x-axis values.
  - `hue_column` (str): Column name in data for the hue/categorical variable.
  - `ax` (matplotlib.axes.Axes, optional): Axes to plot on. If None, uses the current axes. Defaults to None.


#### Steps:
1. **Get Y-Axis Limits**: Retrieve the current y-axis limits.
2. **Iterate Over Categories**: For each unique category in the hue column:
   - Identify the minimum and maximum values.
   - Calculate the y-offset for the text to ensure it stays within plot boundaries.
   - Annotate the minimum value with a custom arrow style.
   - Annotate the maximum value with a different custom arrow style.
3. **Add Legend**: For the symbols used in annotations.


In [ ]:
# Apply a consistent visual theme throughout the project
theme = load_theme("boxy_dark")
theme.apply()

# Define standard dimensions for plots
standard_plot_width = 12
standard_plot_height = 5

# Define a list of markers for each category
markers = ['o', 's', 'D', '^', 'v', '<', '>', 'p', '*', 'h', 'H', '+', 'x', 'D', 'd', 'P', 'X']


In [ ]:
#Plotting functions

def plot_display(title, xlabel, ylabel, legendlabel=None, data=None, ycolumn=None, xcolumn=None, huecolumn=None, xticksrotation=45,
                 legendlocation='upper left', legendanchor=(1, 1), legendboxpadding=1, ax=None, format_datetime=False):

    show_plot = False # If axes are used, show plt.show() & plt.tight_layout() to be called externally
    if ax is None:
        ax = plt.gca()  # Get the current axes if none is provided
        show_plot = True

    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.tick_params(axis='x', rotation=xticksrotation)

    # Format x-axis as date-time if specified
    if format_datetime:
        ax.xaxis.set_major_locator(mat_dates.MonthLocator(interval=1))  # One tick per month
        ax.xaxis.set_major_formatter(mat_dates.DateFormatter('%b-%Y'))  # Format as 'Jan-2020'

    # Check if data is a DataFrame and not empty & Annotate Min Max Values
    if isinstance(data, pd.DataFrame) and not data.empty and ycolumn is not None and xcolumn is not None:
        annotate_min_max(data, ycolumn, xcolumn, hue_column=huecolumn, ax=ax)

    if legendlabel is not None:
        ax.legend(title=legendlabel, loc=legendlocation, bbox_to_anchor=legendanchor, borderaxespad=legendboxpadding,
                  fancybox=1, borderpad=1, fontsize='small')

    if show_plot:
        plt.tight_layout()
        plt.show()


def annotate_min_max(data, yaxisdata, xasixdata, hue_column, ax=None):

    if ax is None:
        ax = plt.gca() # Get the current axes if none is provided

    # Get the y-axis limits
    y_min, y_max = ax.get_ylim()

    # Iterate over each unique category in the hue column
    for category in data[hue_column].unique():
        subset = data[data[hue_column] == category]

        min_idx = subset[yaxisdata].idxmin()
        max_idx = subset[yaxisdata].idxmax()

        min_val = subset.loc[min_idx, yaxisdata]
        max_val = subset.loc[max_idx, yaxisdata]
        min_date = subset.loc[min_idx, xasixdata]
        max_date = subset.loc[max_idx, xasixdata]

        # Calculate the y-offset for the text
        y_offset = 0.05 * (y_max - y_min)

        # Ensure the text stays within the plot boundaries for the y-axis
        min_text_y = min(min_val + y_offset, y_max - y_offset)
        max_text_y = max(max_val - y_offset, y_min + y_offset)

        # Annotate the minimum value with a custom arrow style
        ax.annotate(f'{min_val:.2f}',
                     xy=(min_date, min_val),
                     xytext=(min_date, min_text_y),
                     textcoords="data",
                     arrowprops=dict(arrowstyle='->,head_length=0.5,head_width=0.5',
                                     facecolor='red',
                                     edgecolor='red',
                                     linewidth=2,
                                     linestyle='--'),
                     fontsize=9, color='red',
                     horizontalalignment='center')

        # Annotate the maximum value with a different custom arrow style
        ax.annotate(f'{max_val:.2f}',
                     xy=(max_date, max_val),
                     xytext=(max_date, max_text_y),
                     textcoords="data",
                     arrowprops=dict(arrowstyle='-[,widthB=0.5,lengthB=0.3',
                                     facecolor='green',
                                     edgecolor='green',
                                     linewidth=2,
                                     linestyle='-'),
                     fontsize=9, color='green',
                     horizontalalignment='center')

    # Add a legend for the symbols
    ax.plot([], [], 'r>', label='Lowest', linewidth=2)
    ax.plot([], [], 'g-', label='Highest', linewidth=2)
    ax.legend(loc='upper right')



# Section 7: Data Analysis (National Categories)

## Appointments per Month by Service Setting

## Analysis

- **Data Grouping**: Grouped by month and service setting to sum appointments.
- **Visualization**: Uses unique colors and markers for each service setting.
- **Line Plot**: Shows appointment trends by service setting.

## Insights

- **Trends**: "General Practice" has the highest appointments, peaking in October 2021 and March 2022. Other categories like "Extended Access Provision" and "Primary Care Network" have low numbers.
- The other categories are expanded in the next graph


In [ ]:
# Group data by month and service setting (ss) to sum appointments
nc_month_service_app = nc.groupby(['appointment_month_year', 'service_setting'], as_index=False, observed=True)[
    'count_of_appointments'].sum()

# Extract unique service settings for consistent coloring
unique_service_settings = nc_month_service_app['service_setting'].unique()

# Create a color palette and map each service setting to a color
palette_uss = sns.color_palette("husl", len(unique_service_settings))
color_map_uss = dict(zip(unique_service_settings, palette_uss))

# Create a dictionary to map each service setting to a marker
marker_map_uss = dict(zip(unique_service_settings, markers[:len(unique_service_settings)]))

# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Create line plot for appointment trends by service setting
sns.lineplot(
    data=nc_month_service_app,
    x='appointment_month_year',
    y='count_of_appointments',
    hue='service_setting',
    hue_order=unique_service_settings,
    palette=color_map_uss,
    style='service_setting',
    style_order=unique_service_settings,
    markers=marker_map_uss,
    dashes=False,
    errorbar=None
)

# Display plot with title, axis labels, and legend
plot_display(
    title='Appointments per Month by Service Setting',
    xlabel='Month',
    ylabel='Total Appointments',
    legendlabel='Service Setting',
    format_datetime= True
)


# Appointments per Month by Service Setting (Excluding GP)

## Analysis

- **Exclusion**: Filters data to exclude 'General Practice' and focus on other service settings.
- **Visualization**: Creates a line plot for appointment trends by service setting, excluding GP.

## Insights
- **Trends**: The "Unmapped" service setting shows a peak in appointments around October 2021, followed by a decline and stabilization at a lower level. "Primary Care Network", "Extended Access Provision" and "Other" service settings exhibit relatively stable trends with slight fluctuations.


In [ ]:
# Filter the data to exclude 'General Practice' and focus on other service settings
nc_month_service_nongp_app = nc_month_service_app[
    nc_month_service_app['service_setting'] != 'General Practice']

# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Extract unique service settings excluding GP for consistent coloring
unique_service_settings_nongp = nc_month_service_nongp_app['service_setting'].unique()

# Create line plot for appointment trends by service setting excluding GP
sns.lineplot(
    data=nc_month_service_nongp_app,
    x='appointment_month_year',
    y='count_of_appointments',
    hue='service_setting',
    hue_order=unique_service_settings_nongp,
    palette=color_map_uss,
    style='service_setting',
    style_order=unique_service_settings_nongp,
    markers=marker_map_uss,
    dashes=False,
    errorbar=None
)

# Display plot with title, axis labels, and legend
plot_display(
    title='Appointments per Month by Service Setting except GP',
    xlabel='Month',
    ylabel='Total Appointments',
    legendlabel='Service Setting',
    format_datetime= True
)

# Appointments per Month by Context Type

## Analysis

- **Visualization**: A line plot is used to show the trends of appointments per month for each context type.

## Insights

- **Trends**:
  - **Care Related Encounter**: This context type shows the highest number of appointments consistently across all months, with a peak around October 2021, November 2021 and March 2022.


In [ ]:
# Group data by month and context type to sum appointments
nc_ct = nc.groupby(['appointment_month_year', 'context_type'], as_index=False, observed = True)['count_of_appointments'].sum()

# Initialize plot with specified figure size for context type trends
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Extract unique service settings for consistent coloring
unique_context_type = nc_ct['context_type'].unique()

# Create a dictionary to map each context type to a marker
marker_map_uct = dict(zip(unique_context_type, markers[:len(unique_context_type)]))

# Create line plot for appointment trends by context type
sns.lineplot(
    data=nc_ct,
    x='appointment_month_year',
    y='count_of_appointments',
    hue='context_type',
    hue_order=unique_context_type,
    style='context_type',
    style_order=unique_context_type,
    markers=marker_map_uct,
    dashes=False,
    errorbar=None
)

# Display plot with title, axis labels, and legend
plot_display(
    title='Appointments per Month by Context Type',
    xlabel='Month',
    ylabel='Total Appointments',
    legendlabel='Context Type',
    format_datetime= True
)



# Appointments per Month by National Categories (Top 7)

## Analysis

- **Data Grouping**: Grouped by month and national category to sum appointments.
- **Top Categories**: Focuses on the top 7 national categories with the highest total appointments.
- **Visualization**: Uses unique colors and markers for each national category.
- **Line Plot**: Shows appointment trends by national category.

## Insights

- **Trends**: The top 7 national categories exhibit varying trends in appointment numbers over time. General consultation routine had the highest appointment volume, followed by general consultation acute and clinical triage.Some categories may show peaks at specific months, while others remain relatively stable.


In [ ]:
# Group data by month and national category to sum appointments
nc_nc = nc.groupby(['appointment_month_year', 'national_category'], as_index=False, observed = True)['count_of_appointments'].sum()

# Calculate the total sum of appointments for each national category
category_sums_nc = nc_nc.groupby(['national_category'],observed = True)['count_of_appointments'].sum()

# Get the top 7 categories with the highest total appointments
top_7_categories_nc = category_sums_nc.nlargest(7).index

# Filter the DataFrame to include only the top 7 categories
nc_nc_top7 = nc_nc[nc_nc['national_category'].isin(top_7_categories_nc)]

# Extract unique national categories from the filtered dataset for consistent coloring
unique_national_categories = nc_nc_top7['national_category'].unique()

# Create a dictionary to map each context type to a marker
marker_map_unc = dict(zip(unique_national_categories, markers[:len(unique_national_categories)]))

# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Create line plot for appointment trends by top 7 national categories
sns.lineplot(
    data=nc_nc_top7,
    x='appointment_month_year',
    y='count_of_appointments',
    hue='national_category',
    hue_order=unique_national_categories,
    style='national_category',
    style_order=unique_national_categories,
    markers=marker_map_unc,
    dashes=False,
    errorbar=None
)

# Display plot with title, axis labels, and legend
plot_display(
    title='Appointments per Month by National Categories (Top 7)',
    xlabel='Month',
    ylabel='Total Appointments',
    legendlabel='National Categories',
    format_datetime= True
)

# Commented out code for plotting all categories (too many categories)
# plt.figure(figsize=(standard_plot_width, standard_plot_height))
# sns.lineplot(
#     data=nc_nc,
#     x='appointment_month_year',
#     y='count_of_appointments',
#     hue='national_category',
#     errorbar=None
# )


# Appointments per Month (Appointments Regional)

## Analysis

- **Data Grouping**: Grouped by appointment month to sum the count of appointments.
- **Visualization**: Uses a line plot to show the trend of appointments over time.
- **Line Plot**: Displays the total number of appointments per month.

## Insights

- **Trends**:
  - **Early 2020**: There is a noticeable decline in appointments from January 2020 to April 2020, likely influenced by the onset of the COVID-19 pandemic.
  - **Mid-2020 Recovery**: A recovery in appointment numbers begins around May 2020, with a steady increase peaking in October 2020.
  - **Fluctuations in 2021**: Throughout 2021, the number of appointments fluctuates, with notable peaks in March and August, and a significant dip in July.
  - **Late 2021 to Mid-2022**: The trend continues with fluctuations, showing peaks in October 2021, January 2022, and April 2022, followed by a decline towards June 2022.



In [ ]:
# Group data by appointment month to sum the count of appointments
ar_month_app = ar.groupby('appointment_month_year', as_index=False)['count_of_appointments'].sum()

# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Create line plot for appointment trends by month
sns.lineplot(
    data=ar_month_app,
    x='appointment_month_year',
    y='count_of_appointments',
    errorbar=None,
    marker='.'
)

# Display plot with title, axis labels, and no legend
plot_display(
    title='Appointments per Month',
    xlabel='Month',
    ylabel='Total Appointments',
    legendlabel=None,
    format_datetime= True
)


# Appointment Status Analysis (Post-August 2021)

## Overview

- **Data Set Period**: 1st January 2020 - 1st June 2022
- **COVID Lockdown Period**: March 2020 - July 2021
- **COVID Cases in UK**: Began in January 2020

## Analysis Periods

- **COVID Period**: January 2020 - July 2021
- **Post-COVID Period**: August 2021 - June 2022

## Reason for Data Selection

The data is filtered to include only appointments from August 1, 2021, onwards to focus on the post-COVID period. This allows for an analysis of appointment trends and statuses after the major COVID lockdowns and restrictions, providing insights into how appointment patterns after the COVID outlier period .

## Analysis

- **Data Filtering**: The DataFrame is filtered to include only appointments from August 1, 2021, onwards.
- **Data Grouping**: Grouped by appointment status to sum the count of appointments.
- **Visualization**: Uses a bar plot to show the total number of appointments by appointment status.

## Insights

- **Appointment Status Trends**:
  - **Attended**: The vast majority of appointments fall under the "Attended" status, indicating that most scheduled appointments were successfully attended by patients.
  - **DNA (Did Not Attend)**: A smaller portion of appointments were marked as "DNA," where patients did not attend their scheduled appointments. Number of these appointments over an 11-month period, totaling 13,318,384 appointments—averaging 1,210,762 per month.


In [ ]:
# Filter the DataFrame to include only appointments from August 1, 2021, onwards
ar_time_aft082021 = ar[ar['appointment_month_year'] >= pd.Timestamp('2021-08-01')]

# Group the filtered data by appointment status and sum the count of appointments
ar_as_app = ar_time_aft082021.groupby(['appointment_status'], observed = True)['count_of_appointments'].sum().reset_index()

# Initialize plot
plt.figure(figsize=(7, standard_plot_height))

# Create a bar plot for appointment status
sns.barplot(
    data=ar_as_app,
    x='appointment_status',
    y='count_of_appointments',
    errorbar=None,
    hue = 'appointment_status',
    palette='viridis',
    width=0.5  # Reduce the width of the bars
)

# Display plot with title, axis labels, and no legend
plot_display(
    title='Appointment Status',
    xlabel='Appointment Status',
    ylabel='Total Appointments',
    legendlabel=None
)


# Appointments per Month by Healthcare Professional Type

## Analysis

- **Data Grouping**: Grouped by month and healthcare professional type to sum the count of appointments.
- **Visualization**: Uses a line plot to show the trend of appointments over time for each healthcare professional type.
- **Line Plot**: Displays the total number of appointments per month, categorized by healthcare professional type.

## Insights

- **Trends by Healthcare Professional Type**:
  - **GP (General Practitioner)**: This category consistently shows the highest number of appointments across all months. There are fluctuations, with notable peaks around November 2021 and March 2022.
  - **Other Practice Staff**: This category shows a steady number of appointments, generally lower than GPs but with similar trends and fluctuations over time.


In [ ]:
# How do the healthcare professional types differ over time?

# Group the data by month and healthcare professional type to sum the count of appointments
ar_month_app_hcp = ar_time_aft082021.groupby(['appointment_month_year', 'hcp_type'], as_index=False, observed=True)[
    'count_of_appointments'].sum()

# Extract unique national categories from the filtered dataset for consistent coloring
unique_hcp_types = ar_month_app_hcp['hcp_type'].unique()

# Create a dictionary to map each context type to a marker
marker_map_uhc = dict(zip(unique_hcp_types, markers[:len(unique_hcp_types)]))

# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Create a line plot for appointment trends by month and healthcare professional type
sns.lineplot(
    data=ar_month_app_hcp,
    x='appointment_month_year',
    y='count_of_appointments',
    hue='hcp_type',
    hue_order=unique_hcp_types,
    style='hcp_type',
    style_order=unique_hcp_types,
    markers=marker_map_uhc,
    dashes=False,
    errorbar=None,
)

# Display plot with title, axis labels, and legend
plot_display(
    title='Appointments per Month by Healthcare Professional Type',
    xlabel='Month',
    ylabel='Total Appointments',
    legendlabel='Healthcare Professional Type',
    format_datetime= True
)


# Missed Appointments per Month by Healthcare Professional Type

## Analysis

- **Data Filtering**: The DataFrame is filtered to include only missed appointments (status 'DNA') from August 1, 2021, onwards.
- **Data Grouping**: Grouped by month and healthcare professional type to sum the count of missed appointments.
- **Visualization**: Uses a line plot to show the trend of missed appointments over time for each healthcare professional type.
- **Line Plot**: Displays the total number of missed appointments per month, categorized by healthcare professional type.

## Insights

- **Trends by Healthcare Professional Type**:
  - **Other Practice Staff**: This category shows a noticeable number of missed appointments, with a peak around October 2021 and fluctuations thereafter.
  - **GP (General Practitioner)**: This category also shows a significant number of missed appointments, with trends similar to Other Practice Staff but generally at a slightly lower volume.

In [ ]:
ar_dna_time_aft082021 = ar_time_aft082021[ar_time_aft082021['appointment_status'] == 'DNA']

# Group the data by month and healthcare professional type to sum the count of appointments
ar_dna_month_app_hcp = ar_dna_time_aft082021.groupby(['appointment_month_year', 'hcp_type'], as_index=False, observed=False)[
    'count_of_appointments'].sum()

# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Create a line plot for appointment trends by month and healthcare professional type
sns.lineplot(
    data=ar_dna_month_app_hcp,
    x='appointment_month_year',
    y='count_of_appointments',
    hue='hcp_type',
    hue_order=unique_hcp_types,
    style='hcp_type',
    style_order=unique_hcp_types,
    markers=marker_map_uhc,
    dashes=False,
    errorbar=None
)

# Display plot with title, axis labels, and legend
plot_display(
    title='Missed Appointments per Month by Healthcare Professional Type',
    xlabel='Month',
    ylabel='Missed Appointments',
    legendlabel='Healthcare Professional Type',
    format_datetime= True
)

# Percentage of Missed Appointments per Month by Healthcare Professional Type

## Analysis

- **Data Grouping**: Grouped by month and healthcare professional type to calculate the total and missed (DNA) appointments.
- **Data Merging**: Merged the total and missed appointment counts to calculate the percentage of missed appointments.
- **Visualization**: Uses a line plot to show the trend of the percentage of missed appointments over time for each healthcare professional type.
- **Line Plot**: Displays the percentage of missed appointments per month, categorized by healthcare professional type.


In [ ]:
# 1. Group by month and HCP type to get total appointments
total_by_hcp_type = ar_time_aft082021.groupby(['appointment_month_year', 'hcp_type'])['count_of_appointments'].sum().reset_index(name='total')

# 2. Group by month and HCP type to get missed (DNA) appointments
missed_by_hcp_type = ar_time_aft082021[ar_time_aft082021['appointment_status'] == 'DNA'].groupby(['appointment_month_year', 'hcp_type'])['count_of_appointments'].sum().reset_index(name='missed')

# 3. Merge the total and missed counts
summary_hcp_type = pd.merge(total_by_hcp_type, missed_by_hcp_type, on=['appointment_month_year', 'hcp_type'], how='left')
summary_hcp_type['missed'] = summary_hcp_type['missed'].fillna(0)

# 4. Calculate percentage missed, only if total is not zero
summary_hcp_type['percent_missed'] = np.where(
    summary_hcp_type['total'] != 0,
    (summary_hcp_type['missed'] / summary_hcp_type['total']) * 100,
    0  # Default value if total is zero
)
# Create a color palette and map each HCP type to a color
palette_sum_hcp = sns.color_palette("husl", len(unique_hcp_types))
color_ma_sum_hcp = dict(zip(unique_hcp_types, palette_sum_hcp))

# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Create line plot for missed appointment trends by HCP type
sns.lineplot(
    data=summary_hcp_type,
    x='appointment_month_year',
    y='percent_missed',
    hue='hcp_type',
    hue_order=unique_hcp_types,
    palette=color_ma_sum_hcp,
    style='hcp_type',
    style_order=unique_hcp_types,
    markers=marker_map_uhc,
    dashes=False,
    errorbar=None
)

# Display plot with title, axis labels, and legend
plot_display(
    title='% Missed Appointments per Month by Healthcare Professional Type',
    xlabel='Month',
    ylabel='Missed Appointment %',
    legendlabel='Healthcare Professional Type',
    data=summary_hcp_type,
    ycolumn='percent_missed',
    xcolumn='appointment_month_year',
    huecolumn='hcp_type',
    format_datetime= True
)

- **Trends by Healthcare Professional Type**:
  - **Other Practice Staff**: This category also shows fluctuations, with a noticeable peak in missed appointments around October & December 2021 and subsequent variations.
  -  Other practice staff had more than double the DNA rate compared to GPs and unknown categories.

# Total Appointments per Month by Appointment Mode

## Analysis

- **Data Grouping**: Grouped by month and appointment mode to sum the count of appointments.
- **Visualization**: Uses a line plot to show the trend of appointments over time for each appointment mode.
- **Line Plot**: Displays the total number of appointments per month, categorized by appointment mode.

In [ ]:
# 1. Group by month and appointment_mode to get total appointments
total_by_appointment_mode = ar_time_aft082021.groupby(['appointment_month_year', 'appointment_mode'])['count_of_appointments'].sum().reset_index(name='total')

# Create a color palette and map each appointment mode to a color
unique_appointment_modes = total_by_appointment_mode['appointment_mode'].unique()
palette_uam = sns.color_palette("husl", len(unique_appointment_modes))
color_map_uam = dict(zip(unique_appointment_modes, palette_uam))

# Define markers for each appointment mode
marker_map_uam = dict(zip(unique_appointment_modes, markers[:len(unique_appointment_modes)]))

# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Create line plot for appointment trends by appointment mode
sns.lineplot(
    data=total_by_appointment_mode,
    x='appointment_month_year',
    y='total',
    hue='appointment_mode',
    hue_order=unique_appointment_modes,
    palette=color_map_uam,
    style='appointment_mode',
    style_order=unique_appointment_modes,
    markers=marker_map_uam,
    dashes=False,
    errorbar=None
)

# Display plot with title, axis labels, and legend
plot_display(
    title='Total Appointments per Month by Appointment Mode',
    xlabel='Month',
    ylabel='Total Appointments',
    legendlabel='Appointment Mode',
    format_datetime= True
)


## Insights

- **Trends by Appointment Mode**:
  - **Face-to-Face**: This mode consistently shows the highest number of appointments across all months, with noticeable peaks around October 2021 and March 2022.
  - **Telephone**: This mode shows a steady number of appointments, generally lower than face-to-face but with similar trends and fluctuations over time.
  - **Video/Online**: This mode has significantly lower appointment counts compared to face-to-face and telephone, with relatively stable trends and minor fluctuations.
  - **Home Visit** and **Unknown**: These modes have the lowest number of appointments, with relatively stable trends and minor fluctuations over time.

# Missed Appointments per Month by Appointment Mode

## Analysis

- **Data Grouping**: Grouped by month and appointment mode to sum the count of missed (DNA) appointments.
- **Visualization**: Uses a line plot to show the trend of missed appointments over time for each appointment mode.
- **Line Plot**: Displays the total number of missed appointments per month, categorized by appointment mode.

## Insights

- **Trends by Appointment Mode**:
  - **Face-to-Face**: This mode shows a noticeable number of missed appointments, with a peak around October 2021 and fluctuations thereafter.
  - **Telephone**: This mode also shows a significant number of missed appointments, with trends similar to face-to-face but generally at a lower volume.
  - **Video/Online**: This mode has significantly lower missed appointment counts compared to face-to-face and telephone, with relatively stable trends and minor fluctuations.
  - **Home Visit** and **Unknown**: These modes have the lowest number of missed appointments, with relatively stable trends and minor fluctuations over time.


In [ ]:
# 1. Group by month and appointment_mode to get missed (DNA) appointments
missed_by_appointment_mode = ar_dna_time_aft082021.groupby(['appointment_month_year', 'appointment_mode'])['count_of_appointments'].sum().reset_index(name='missed')

# Group the data by month and healthcare professional type to sum the count of appointments
ar_dna_month_app_mode = ar_dna_time_aft082021.groupby(['appointment_month_year', 'appointment_mode'], as_index=False, observed=False)[
    'count_of_appointments'].sum()

# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Create a line plot for appointment trends by month and Appointment Mode
sns.lineplot(
    data=ar_dna_month_app_mode,
    x='appointment_month_year',
    y='count_of_appointments',
    hue='appointment_mode',
    hue_order=unique_appointment_modes,
    palette=color_map_uam,
    style='appointment_mode',
    style_order=unique_appointment_modes,
    markers=marker_map_uam,
    dashes=False,
    errorbar=None
)

# Display plot with title, axis labels, and legend
plot_display(
    title='Missed Appointments per Month by Appointment Mode',
    xlabel='Month',
    ylabel='Missed Appointments',
    legendlabel='Appointment Mode',
    format_datetime= True
)

# Percentage of Missed Appointments per Month by Appointment Mode

## Analysis

- **Data Grouping**: Grouped by month and appointment mode to calculate the total and missed (DNA) appointments.
- **Data Merging**: Merged the total and missed appointment counts to calculate the percentage of missed appointments.
- **Visualization**: Uses a line plot to show the trend of the percentage of missed appointments over time for each appointment mode.
- **Line Plot**: Displays the percentage of missed appointments per month, categorized by appointment mode.


In [ ]:

# 1. Merge the total and missed counts
summary_app_mode = pd.merge(total_by_appointment_mode, missed_by_appointment_mode, on=['appointment_month_year', 'appointment_mode'], how='left')
summary_app_mode['missed'] = summary_app_mode['missed'].fillna(0)

# 2. Calculate percentage missed

# 4. Calculate percentage missed, only if total is not zero
summary_app_mode['percent_missed'] = np.where(
    summary_app_mode['total'] != 0,
    (summary_app_mode['missed'] / summary_app_mode['total']) * 100,
    0  # Default value if total is zero
)
# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Create line plot for missed appointment trends by appointment mode
sns.lineplot(
    data=summary_app_mode,
    x='appointment_month_year',
    y='percent_missed',
    hue='appointment_mode',
    hue_order=unique_appointment_modes,
    palette=color_map_uam,
    style='appointment_mode',
    style_order=unique_appointment_modes,
    markers=marker_map_uam,
    dashes=False,
    errorbar=None
)

# Display plot with title, axis labels, and legend
plot_display(
    title='% Missed Appointments per Month by Appointment Mode',
    xlabel='Month',
    ylabel='Missed Appointment %',
    legendlabel='Appointment Mode',
    data=summary_app_mode,
    ycolumn='percent_missed',
    xcolumn='appointment_month_year',
    huecolumn='appointment_mode',
    format_datetime= True
)


- **Trends by Appointment Mode**:
  - **Face-to-Face**: This mode shows fluctuations in the percentage of missed appointments, with a peak around October 2021 and varying trends thereafter.
  - **Telephone**: This mode is relatively stable, showcasing lowest percentage of cancellations.

# Total Appointments per Month by Time Between Booking and Appointment

## Analysis

- **Data Grouping**: Grouped by month and the time between booking and appointment to sum the count of appointments.
- **Visualization**: Uses a line plot to show the trend of appointments over time for each time interval between booking and appointment.
- **Line Plot**: Displays the total number of appointments per month, categorized by the time between booking and appointment.

In [ ]:
# Are there any trends in time between booking and appointment?

# 1. Group by month and time_between_book_and_appointment to get total appointments
total_by_between = ar_time_aft082021.groupby(['appointment_month_year', 'time_between_book_and_appointment'])['count_of_appointments'].sum().reset_index(name='total')

# Create a color palette and map each appointment mode to a color
unique_between = total_by_between['time_between_book_and_appointment'].unique()
palette_utb = sns.color_palette("husl", len(unique_between))
color_map_utb = dict(zip(unique_between, palette_utb))

# Define markers for each appointment mode
marker_map_utb = dict(zip(unique_between, markers[:len(unique_between)]))

# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Create line plot for appointment trends by appointment mode
sns.lineplot(
    data=total_by_between,
    x='appointment_month_year',
    y='total',
    hue='time_between_book_and_appointment',
    hue_order=unique_between,
    palette=color_map_utb,
    style='time_between_book_and_appointment',
    style_order=unique_between,
    markers=marker_map_utb,
    dashes=False,
    errorbar=None
)

# Display plot with title, axis labels, and legend
plot_display(
    title='Total Appointments per Month by time between booking',
    xlabel='Month',
    ylabel='Total Appointments',
    legendlabel='Time between booking and appointment',
    format_datetime= True
)

- **Trends by Time Between Booking and Appointment**:
  - **Same Day**: This category consistently shows the highest number of appointments across all months, indicating that most appointments are booked and attended on the same day.
  - **1 Day**: This category shows a steady number of appointments, generally lower than same-day appointments but with similar trends and fluctuations over time.
  - **2 to 7 Days**: This category has a moderate number of appointments, with fluctuations over time but generally lower than same-day and 1-day categories.
  - **8 to 14 Days**, **15 to 21 Days**, **22 to 28 Days**, and **More than 28 Days**: These categories show relatively lower and stable trends in the number of appointments, with minor fluctuations over time.

# Missed Appointments per Month by Time Between Booking and Appointment

## Analysis

- **Data Grouping**: Grouped by month and the time between booking and appointment to sum the count of missed (DNA) appointments.
- **Visualization**: Uses a line plot to show the trend of missed appointments over time for each time interval between booking and appointment.
- **Line Plot**: Displays the total number of missed appointments per month, categorized by the time between booking and appointment.

## Insights

- **Trends by Time Between Booking and Appointment**:
  - **Same Day**: This category shows fluctuations in the number of missed appointments, with noticeable peaks around October 2021 and varying trends thereafter.
  - **1 Day**: This category also shows fluctuations, generally lower than same-day appointments but with similar trends over time.
  - **2 to 7 Days**: This category has moderate fluctuations in the number of missed appointments, generally lower than same-day and 1-day categories.
  - **8 to 14 Days**, **15 to 21 Days**, **22 to 28 Days**, and **More than 28 Days**: These categories show relatively lower and stable trends in the number of missed appointments, with minor fluctuations over time.

In [ ]:
# 1. Group by month and time_between_book_and_appointment to get missed (DNA) appointments
missed_by_between = ar_dna_time_aft082021.groupby(['appointment_month_year', 'time_between_book_and_appointment'], observed = False)['count_of_appointments'].sum().reset_index(name='missed')

# Group the data by month and healthcare professional type to sum the count of appointments
ar_dna_month_app_between = ar_dna_time_aft082021.groupby(['appointment_month_year', 'time_between_book_and_appointment'], as_index=False, observed=False)[
    'count_of_appointments'].sum()

# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Create a line plot for appointment trends by month and Appointment Mode
sns.lineplot(
    data=ar_dna_month_app_between,
    x='appointment_month_year',
    y='count_of_appointments',
    hue='time_between_book_and_appointment',
    hue_order=unique_between,
    palette=color_map_utb,
    style='time_between_book_and_appointment',
    style_order=unique_between,
    markers=marker_map_utb,
    dashes=False,
    errorbar=None
)

# Display plot with title, axis labels, and legend
plot_display(
    title='Missed Appointments per Month by time between booking and appointment',
    xlabel='Month',
    ylabel='Missed Appointments',
    legendlabel='Time Between Booking and Appointment',
    format_datetime= True
)

# Percentage of Missed Appointments per Month by Time Between Booking and Appointment

## Analysis

- **Data Grouping**: Grouped by month and the time between booking and appointment to calculate the total and missed (DNA) appointments.
- **Data Merging**: Merged the total and missed appointment counts to calculate the percentage of missed appointments.
- **Visualization**: Uses a line plot to show the trend of the percentage of missed appointments over time for each time interval between booking and appointment.
- **Line Plot**: Displays the percentage of missed appointments per month, categorized by the time between booking and appointment.

## Insights

- **Trends by Time Between Booking and Appointment**:
  - **Same Day**: This category shows fluctuations in the percentage of missed appointments, with noticeable peaks and varying trends over time.
  - **1 Day**: This category also shows fluctuations, generally lower than same-day appointments but with similar trends over time.
  - **2 to 7 Days**: This category has moderate fluctuations in the percentage of missed appointments, generally lower than same-day and 1-day categories.
  - **8 to 14 Days**, **15 to 21 Days**, **22 to 28 Days**, and **More than 28 Days**: These categories show relatively lower and stable trends in the percentage of missed appointments, with minor fluctuations over time.

In [ ]:
# 1. Merge the total and missed counts
summary_between = pd.merge(total_by_between, missed_by_between, on=['appointment_month_year', 'time_between_book_and_appointment'], how='left')
summary_between['missed'] = summary_between['missed'].fillna(0)

# 2. Calculate percentage missed, only if total is not zero
summary_between['percent_missed'] = np.where(
    summary_between['total'] != 0,
    (summary_between['missed'] / summary_between['total']) * 100,
    0  # Default value if total is zero
)
# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Create line plot for missed appointment trends by appointment mode
sns.lineplot(
    data=summary_between,
    x='appointment_month_year',
    y='percent_missed',
    hue='time_between_book_and_appointment',
    hue_order=unique_between,
    palette=color_map_utb,
    style='time_between_book_and_appointment',
    style_order=unique_between,
    markers=marker_map_utb,
    dashes=False,
    errorbar=None
)

# Display plot with title, axis labels, and legend
plot_display(
    title='% Missed Appointments per Month by time between booking',
    xlabel='Month',
    ylabel='% Missed Appointments',
    legendlabel='Time Between Booking and Appointment',
    format_datetime= True
)


# Appointments by ICB Code (Descending)

## Analysis

- **Data Grouping**: Grouped by ICB ONS code to sum the count of appointments.
- **Data Sorting**: Sorted in descending order by the total number of appointments.
- **Visualization**: Uses a bar plot to show the total number of appointments for each ICB ONS code.
- **Bar Plot**: Displays the total number of appointments per ICB ONS code, sorted in descending order.

## Insights

- **Distribution by ICB ONS Code**:
  - The bar plot shows a descending distribution of total appointments across various ICB ONS codes.
  - The first few ICB ONS codes have significantly higher appointment counts, indicating a higher volume of appointments in those regions.
  - As we move along the x-axis, the number of appointments gradually decreases, showing a long tail distribution where many ICB ONS codes have relatively lower appointment counts.



In [ ]:
# Sum of count of appointments that were cancelled per icb_ons_code.

# Group by icb_ons_code and sum count_of_appointments
dna_icb = ar_dna_time_aft082021.groupby('icb_ons_code', as_index=False)['count_of_appointments'].sum()

# Sort in descending order by summed appointments
dna_icb_sorted = dna_icb.sort_values('count_of_appointments', ascending=False)

# Initialize plot
plt.figure(figsize=(20, standard_plot_height))

# Create a bar plot
sns.barplot(
    data=dna_icb_sorted,
    x='icb_ons_code',
    y='count_of_appointments',
    errorbar=None,
    order=dna_icb_sorted['icb_ons_code'],
    width=0.8  # Reduce the width of the bars
)

# Display plot with title, axis labels, and no legend
plot_display(
    title='Appointments by ICB Code (Descending)',
    xlabel='ICB ONS Code',
    ylabel='Total Appointments',
    legendlabel= None,
    xticksrotation=90
)

# Percentage of Appointments Missed by ICB Code (Descending)

## Analysis

- **Data Grouping**: Grouped by ICB ONS code to sum the count of appointments and missed appointments.
- **Data Merging**: Merged the total and missed appointment counts to calculate the percentage of missed appointments.
- **Data Sorting**: Sorted in descending order by the percentage of missed appointments.
- **Visualization**: Uses a bar plot to show the percentage of missed appointments for each ICB ONS code.
- **Bar Plot**: Displays the percentage of missed appointments per ICB ONS code, sorted in descending order.

## Insights

- **Distribution by ICB ONS Code**:
  - The bar plot shows a descending distribution of the percentage of missed appointments across various ICB ONS codes.
  - The first few ICB ONS codes have higher percentages of missed appointments, indicating a higher rate of missed appointments in those regions.
  - As we move along the x-axis, the percentage of missed appointments gradually decreases, showing a long tail distribution where many ICB ONS codes have relatively lower percentages of missed appointments.


In [ ]:
# Percentage of appointments cancelled per icb_ons_code
ar_icb = ar_time_aft082021.groupby('icb_ons_code', as_index=False)['count_of_appointments'].sum()

ar_dna_icb = pd.merge(ar_icb, dna_icb, on= 'icb_ons_code', how='inner')

# Calculate percentage missed, only if total is not zero
ar_dna_icb['percent_missed'] = np.where(
    ar_dna_icb['count_of_appointments_x'] != 0,
    (ar_dna_icb['count_of_appointments_y'] / ar_dna_icb['count_of_appointments_x']) * 100,
    0  # Default value if total is zero
)
# Sort in descending order by summed appointments
ar_dna_icb_sorted = ar_dna_icb.sort_values('percent_missed', ascending=False)

# Initialize plot
plt.figure(figsize=(20, standard_plot_height))

# Create a bar plot
sns.barplot(
    data=ar_dna_icb_sorted,
    x='icb_ons_code',
    y='percent_missed',
    errorbar=None,
    order=ar_dna_icb_sorted['icb_ons_code'],
    width=0.8  # Reduce the width of the bars
)

# Display plot with title, axis labels, and no legend
plot_display(
    title='Percentage of Appointments missed by ICB Code (Descending)',
    xlabel='ICB ONS Code',
    ylabel='% Missed Appointments',
    legendlabel= None,
    xticksrotation=90
)

# Service Setting Distribution by Season

## Analysis

- **Data Grouping**: Grouped by service setting and season/year to sum the count of appointments.
- **Visualization**: Uses a series of line plots to show the distribution of appointments across different service settings for each season.
- **Subplots**: Displays four subplots, each representing a different season (Summer 2021, Autumn 2021, Winter 2021-2022, Spring 2022).

## Insights

- **Seasonal Trends**:
  - **Summer 2021**: The distribution shows a peak in appointments for "General Practice," with relatively lower counts for other service settings.
  - **Autumn 2021**: Similar to Summer 2021, "General Practice" has the highest number of appointments, with a noticeable peak.
  - **Winter 2021 - 2022**: This season also shows a significant peak in "General Practice" appointments, with other service settings having lower counts.
  - **Spring 2022**: The trend continues with "General Practice" having the highest appointments, followed by other service settings with lower counts.

In [ ]:
# Group data by service setting and season/year, summing appointment counts
nc_ss_day = nc.groupby(['service_setting', 'season_year'], as_index=False)['count_of_appointments'].sum()

# Create a figure with 2x2 subplots
fig, axes = plt.subplots(2, 2, figsize=(standard_plot_width, standard_plot_height*1.8), sharex=True, sharey=True)
axes = axes.flatten()

# Define seasons for plotting
seasons = ['Summer 2021', 'Autumn 2021', 'Winter 2021 - 2022', 'Spring 2022']

# Plot line charts for each season
for i, season in enumerate(seasons):
    season_data = nc_ss_day[nc_ss_day['season_year'] == season]
    sns.lineplot(
        data=season_data,
        x='service_setting',
        y='count_of_appointments',
        ax=axes[i],
        errorbar=None
    )
    plot_display(
        title=f"Service Setting distribution for {season}",
        xlabel='Service Setting',
        ylabel='Total Appointments',
        legendlabel=None,
        ax=axes[i]  # Use the current subplot axis
    )

plt.tight_layout()
plt.show()


# Appointment Distribution for General Practice (GP)

## Analysis

- **Data Grouping**: Grouped by appointment month and service setting to sum the count of appointments, specifically filtering for "General Practice."
- **Visualization**: Uses a line plot to show the trend of appointments over time for General Practice.
- **Line Plot**: Displays the total number of appointments per month for General Practice, with the y-axis limited to a range between 150,000 and 30,000,000.

## Insights

- **Trends**:
  - **August 2021 to October 2021**: There is a noticeable increase in the number of appointments, peaking around October 2021.
  - **November 2021 to January 2022**: A decline in appointments is observed, reaching a lower point around January 2022.
  - **February 2022 to April 2022**: The number of appointments rises again, with another peak around March 2022.
  - **May 2022 to June 2022**: The trend shows a slight decline in appointments towards June 2022.

In [ ]:
# Group data by appointment month to sum the count of appointments
nc_month_app = nc.groupby(['appointment_month_year', 'service_setting'], as_index=False)['count_of_appointments'].sum()
nc_month_app_gp = nc_month_app[nc_month_app['service_setting'] == 'General Practice']
# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Create line plot for appointment trends by month
axes_nc_month_app_gp = sns.lineplot(
    data=nc_month_app_gp,
    x='appointment_month_year',
    y='count_of_appointments',
    errorbar=None,
    marker='.'
)
axes_nc_month_app_gp.set_ylim(150000, 30000000)
# Display plot with title, axis labels, and no legend
plot_display(
    title='Appointment distribution for GP',
    xlabel='Month',
    ylabel='Total Appointments',
    legendlabel=None,
    format_datetime= True
)


# Total Appointments by Actual Duration

## Analysis

- **Data Grouping**: Grouped by actual duration to sum the count of appointments.
- **Visualization**: Uses a line plot to show the distribution of appointments across different actual durations.
- **Line Plot**: Displays the total number of appointments for each actual duration category.

## Insights

- **Trends by Actual Duration**:
  - **1-5 Minutes**: This category shows a moderate number of appointments.
  - **6-10 Minutes**: This category has the highest number of appointments, indicating that most appointments fall within this duration range.
  - **11-15 Minutes**: This category shows a noticeable decline in the number of appointments compared to the 6-10 minutes category.
  - **16-20 Minutes**, **21-30 Minutes**, and **31-60 Minutes**: These categories show a further decline in the number of appointments, with relatively stable trends.
  - **Unknown / Data Quality**: This category has the highest number of appointments, indicating potential data quality issues

In [ ]:
# Group data by appointment month to sum the count of appointments
ad_ad = ad.groupby(['actual_duration'], as_index=False)['count_of_appointments'].sum()

# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Create line plot for appointment trends by month
sns.lineplot(
    data=ad_ad,
    x='actual_duration',
    y='count_of_appointments',
    errorbar=None,
    marker='.'
)

# Display plot with title, axis labels, and no legend
plot_display(
    title='Total Appointments by Actual Duration',
    xlabel='Actual Duration',
    ylabel='Total Appointments',
    legendlabel=None
)


 # Section 8: Further Analysis with Resources Data

## Overview

- **Data Streamlining**: Focus on General Practitioners (GPs) within the `nc` dataset.
- **Data Combination**: Merge `nc` and `rd` datasets for comprehensive analysis.
- **Time Frame**: Consider data from August 2021 onwards, as all lockdowns were lifted by July 2021.

## Analysis Steps

### Data Grouping and Combination

1. **Group Data by Month and ICB ONS Code**:
   - Group `nc` data by month, ICB ONS code, and count of appointments.
   - Group `rd` data by month, ICB ONS code, Full-Time Equivalent (FTE) total, and employee count.
   - Combine the grouped data from `nc` and `rd` for further analysis.

2. **Visualization**:
   - Draw graphs to analyze the combined data, focusing on appointment counts, FTE totals, and employee counts.

### Comparison with DNA Data

- **Data Limitation**: Unable to directly compare with DNA data as the `ar` dataframe is not limited to GPs. However as seen in previous graphs, GP constitute 91.5% of the total appointments.
- **Pattern Analysis**:
  - Look up patterns for various points in the DNA dataframe.
  - Identify ICBs with the highest missed appointments.
  - For identified ICBs, check the number of GP appointments in the `nc` dataframe.
  - Compare with staff and capacity utilization in those ICBs.

### Deep Dive Analysis

- **ICB Focus**: For ICBs that come into attention, perform a deep dive to identify staff shortages.
- **Utilization Rate Calculation**:
  - Calculate utilization rates for weekdays separately, as averaging including weekends may give inaccurate results.

### Recommendations to Reduce Missed Appointments

- **Percentage Analysis**:
  - Deep dive based on the percentage of missed appointments for ICBs.
  - Focus on the top 10 ICBs with the highest missed appointment percentages.
  - Provide granular details and recommendations for these specific ICBs.

- **Efficiency Analysis**:
  - Use actual duration data to check the efficiency of sub-ICBs.
  - Analyze GP appointment percentages and staff availability for these sub-ICBs.


In [ ]:
#Merge with sub-icb summary document to get the column for combining with appointments regional and keep only icb codes that are present in the provided database. There was 1 additional icb data that got dropped off
rd = pd.merge(sub_icb_summary, rgp, on= 'sub_icb_location_code', how='left')
#subset dataframe for only GP service setting.
nc_gp = nc[nc['service_setting'] == 'General Practice']


In [ ]:
nc_gp_group = nc_gp.groupby(['appointment_month_year'])['count_of_appointments'].sum().reset_index()
rd_group = rd.groupby(['appointment_month_year'])[['fte_total', 'employee_count']].sum().reset_index()
nc_rd_gp = pd.merge(nc_gp_group, rd_group, on=['appointment_month_year'], how='left')

# Monthly Count of Appointments and Resources

## Analysis

- **Data Visualization**: Uses bar charts to show the monthly count of appointments and the monthly employee count and Full-Time Equivalent (FTE) total.
- **Subplots**: Displays two subplots, one for the monthly count of appointments and another for the monthly employee count and FTE total.

## Insights

### Monthly Count of Appointments

- **Trends**:
  - The monthly count of appointments shows fluctuations over the observed period.
  - Peaks in appointment counts are observed around October 2021 and March 2022.
  - There are noticeable declines in appointment counts around December 2021 and May 2022.

### Monthly Employee Count and FTE Total

- **Employee Count**:
  - The employee count remains relatively stable over the observed period, with minor fluctuations.
  - The highest employee counts are observed around October 2021 and March 2022.

- **FTE Total**:
  - The FTE total also shows stability with minor fluctuations over the observed period.
  - Peaks in FTE totals are observed around October 2021 and March 2022, aligning with the peaks in employee counts.


In [ ]:
# Create x-axis positions
x_labels_nc_rd_gp = nc_rd_gp['appointment_month_year'].dt.strftime('%b-%Y')
x_pos = np.arange(len(x_labels_nc_rd_gp))  # [0, 1, 2, ..., n-1]

# Set up plot with 2 subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(standard_plot_width, standard_plot_height*1.8), sharex=True)

# First bar chart: Monthly Count of Appointments
bars1 = ax1.bar(
    x_pos,
    nc_rd_gp['count_of_appointments'],
    color='steelblue')

# Add values above bars in first plot
for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2, height, f'{int(height)}',
             ha='center', va='bottom', fontsize=9)

# Second grouped bar chart: Monthly Employee Count and FTE Total
bar_width = 0.4
bars2_emp = ax2.bar(
    x_pos - bar_width/2,
    nc_rd_gp['employee_count'],
    width=bar_width,
    color='seagreen',
    label='Employee Count')

bars2_fte = ax2.bar(
    x_pos + bar_width/2,
    nc_rd_gp['fte_total'],
    width=bar_width,
    color='orange',
    label='FTE Total')

# Add values above bars in second plot
for bar in bars2_emp:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2, height, f'{height:.1f}',
             ha='center', va='bottom', fontsize=9)

for bar in bars2_fte:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2, height, f'{height:.1f}',
             ha='center', va='bottom', fontsize=9)

# Set x-ticks and labels correctly for both subplots
ax2.set_xticks(x_pos)
ax2.set_xticklabels(x_labels_nc_rd_gp, rotation=90)

# Remove top spine to avoid overlapping with bar labels
ax1.spines['top'].set_visible(False)
ax2.spines['top'].set_visible(False)

# Display plot with title, axis labels, and no legend
plot_display(
    title='Monthly Count of Appointments',
    xlabel='Months',
    ylabel='Total Appointments',
    legendlabel=None,
    ax=ax1
)

plot_display(
    title='Monthly Employee Count and FTE Total',
    xlabel='Months',
    ylabel='Total Appointments',
    legendlabel="Resources",
    ax=ax2
)

plt.tight_layout()
plt.show()


# Number of Cancelled Appointments per Month

## Analysis

- **Data Grouping**: Grouped by appointment month to sum the count of canceled appointments.
- **Visualization**: Uses a line plot to show the trend of canceled appointments over time.
- **Line Plot**: Displays the total number of canceled appointments per month.

## Insights

- **Trends**:
  - **August 2021 to October 2021**: There is a noticeable increase in the number of canceled appointments, peaking around October 2021.
  - **November 2021 to January 2022**: A decline in canceled appointments is observed, reaching a lower point around January 2022.
  - **February 2022 to April 2022**: The number of canceled appointments rises again, with another peak around March 2022.
  - **May 2022 to June 2022**: The trend shows a slight decline in canceled appointments towards June 2022.

In [ ]:
ar_dna_month_app = ar_dna_time_aft082021.groupby('appointment_month_year', as_index=False)['count_of_appointments'].sum()
# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Create line plot for appointment trends by month
sns.lineplot(
    data=ar_dna_month_app,
    x='appointment_month_year',
    y='count_of_appointments',
    errorbar=None,
    marker='.'
)

# Display plot with title, axis labels, and no legend
plot_display(
    title='Number of cancelled Appointments per Month',
    xlabel='Month',
    ylabel='Total Appointments',
    format_datetime= True
)

# Analysis of Staff Groups within Resources Data

## Overview

- **Focus on FTE**: We will focus on Full-Time Equivalent (FTE) data, as it provides a clearer picture of the actual working staff.
- **Data Segmentation**: Within the resources database (`rd`), there is a column for staff groups. We will split this data into separate datasets to evaluate each staff group individually.


In [ ]:
# Group data for GPs by appointment month/year, summing FTE total and employee count
rd_gp_group = rd[rd['staff_group'] == 'GP'].groupby(['appointment_month_year','staff_group'])[['fte_total', 'employee_count']].sum().reset_index()

# Group data for Nurses by appointment month/year, summing FTE total and employee count
rd_nurse_group = rd[rd['staff_group'] == 'Nurses'].groupby(['appointment_month_year','staff_group'])[['fte_total', 'employee_count']].sum().reset_index()

# Group data for Direct Patient Care by appointment month/year, summing FTE total and employee count
rd_dpc_group = rd[rd['staff_group'] == 'Direct Patient Care'].groupby(['appointment_month_year','staff_group'])[['fte_total', 'employee_count']].sum().reset_index()

# Group data for Admin/Non-Clinical by appointment month/year, summing FTE total and employee count
rd_admin_group = rd[rd['staff_group'] == 'Admin/Non-Clinical'].groupby(['appointment_month_year','staff_group'])[['fte_total', 'employee_count']].sum().reset_index()




# FTE per Employee Over Time for Different Staff Groups

## Analysis

- **Data Calculation**: Calculated the FTE per employee ratio for each staff group (GP, Nurses, DPC, Admin/Non-Clinical).
- **Visualization**: Uses line plots to show the trend of the FTE per employee ratio over time for each staff group.
- **Subplots**: Displays four subplots, each representing a different staff group (GP, Nurses, DPC, Admin/Non-Clinical).

## Insights

### FTE per Employee Trends
- The FTE remains stable across all groups, consistently indicating that only 70-75% of the total employees are working as full-time equivalents.
- That suggests that a significant portion of the workforce is not engaged in full-time roles.

In [ ]:
# Calculate FTE per employee ratio for each group
rd_gp_group['fte_per_employee'] = rd_gp_group['fte_total'] / rd_gp_group['employee_count']
rd_nurse_group['fte_per_employee'] = rd_nurse_group['fte_total'] / rd_nurse_group['employee_count']
rd_dpc_group['fte_per_employee'] = rd_dpc_group['fte_total'] / rd_dpc_group['employee_count']
rd_admin_group['fte_per_employee'] = rd_admin_group['fte_total'] / rd_admin_group['employee_count']

# Create a figure with 2x2 subplots
fig, axes = plt.subplots(2, 2, figsize=(standard_plot_width, standard_plot_height*1.8), sharex=False)
axes = axes.flatten()

# Define staff groups for plotting with color and marker maps
staff_groups = [
    ('GP', rd_gp_group, 'purple', 'o'),
    ('Nurses', rd_nurse_group, 'blue', 's'),
    ('DPC', rd_dpc_group, 'green', '^'),
    ('Admin/Non-Clinical', rd_admin_group, 'orange', 'D')
]

# Plot line charts for each staff group
for i, (label, data_staff_groups, color, marker) in enumerate(staff_groups):
    axes_staff_group = sns.lineplot(
        data=data_staff_groups,
        x='appointment_month_year',
        y='fte_per_employee',
        ax=axes[i],
        marker=marker,
        color=color,
        errorbar=None
    )
    axes_staff_group.set_ylim(0, 1)

    plot_display(
        title=f"FTE per Employee Over Time for {label}",
        xlabel='Months',
        ylabel='FTE / Employee',
        legendlabel=None,
        data=data_staff_groups,
        ycolumn='fte_per_employee',
        xcolumn='appointment_month_year',
        huecolumn='staff_group',
        format_datetime=True,
        ax=axes[i]  # Use the current subplot axis
    )

plt.tight_layout()
plt.show()

# Daily Appointments per Staff Type

## Analysis

- **Data Merging and Calculation**: Merged data from different staff types and calculated the number of appointments per staff per day.
- **Visualization**: Uses a line plot to show the trend of daily appointments per Full-Time Equivalent (FTE) for each staff type over time.
- **Line Plot**: Displays the daily appointments per FTE for each staff type, with different colors and markers representing each staff group.

## Insights

- **Trends by Staff Type**:
- It shows nurses and Direct Patient Care resources have significantly more appointments load daily compared to Admin/ General Practitioners

In [ ]:
# Initialize plot with specified figure size for staff type trends
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Merge data and calculate appointments per staff per day for each staff type
staff_types = [
    ('GP', nc_gp_group, rd_gp_group),
    ('Nurses', nc_gp_group, rd_nurse_group),
    ('DPC', nc_gp_group, rd_dpc_group),
    ('Admin/Non-Clinical', nc_gp_group, rd_admin_group)
]
# Initialize dictionaries for color and marker maps
color_map_rd = {}
marker_map_rd = {}

# Populate the dictionaries
for group in staff_groups:
    label, _, color, marker = group
    color_map_rd[label] = color
    marker_map_rd[label] = marker

# Initialize plot with specified figure size for staff type trends
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Plot appointments per staff per day for each staff type
for label, nc_group, rd_group in staff_types:
    nc_rd_staff = pd.merge(nc_group, rd_group, on=['appointment_month_year'], how='left')
    nc_rd_staff['appts_per_staff'] = nc_rd_staff['count_of_appointments'] / nc_rd_staff['fte_total']
    nc_rd_staff['appts_per_staff_per_day'] = nc_rd_staff['appts_per_staff'] / 30

    sns.lineplot(
        data=nc_rd_staff,
        x='appointment_month_year',
        y='appts_per_staff_per_day',
        marker=marker_map_rd[label],
        color= color_map_rd[label],
        label=label,
        errorbar=None
    )

# Display plot with title, axis labels, and legend
plot_display(
    title='Daily Appointments per Staff Type ',
    xlabel='Appointment Month-Year',
    ylabel='Appointments per FTE',
    legendlabel='Staff Group',
    format_datetime=True
)
print("Daily average Appointments per Staff Type:")

# Print mean appointments per staff per day for each staff type
for label, nc_group, rd_group in staff_types:
    nc_rd_staff = pd.merge(nc_group, rd_group, on=['appointment_month_year'], how='left')
    nc_rd_staff['appts_per_staff'] = nc_rd_staff['count_of_appointments'] / nc_rd_staff['fte_total']
    nc_rd_staff['appts_per_staff_per_day'] = nc_rd_staff['appts_per_staff'] / 30
    print(f"{label}: {nc_rd_staff['appts_per_staff_per_day'].mean().round(1)}")


# Number of Appointments by Day of the Week

## Analysis

- **Data Grouping**: Grouped by the day of the week to sum the count of appointments.
- **Visualization**: Uses a bar plot to show the total number of appointments for each day of the week.
- **Bar Plot**: Displays the total number of appointments per day of the week.

## Insights

- **Trends by Day of the Week**:
  - **Monday to Friday**: These days show a high and relatively consistent number of appointments, with Monday through Thursday having the highest counts.
  - **Friday**: Shows a slight decrease in the number of appointments compared to Monday through Thursday.
  - **Saturday & Sunday**: There is a significant drop in the number of appointments on Saturday.

In [ ]:
# Initialize plot
plt.figure(figsize=(8, standard_plot_height))

nc_day = nc.groupby(['appointment_day_of_the_week'])['count_of_appointments'].sum().reset_index()

# Create a bar plot
sns.barplot(
    data=nc_day,
    x='appointment_day_of_the_week',
    y='count_of_appointments',
    errorbar=None,
    hue = 'appointment_day_of_the_week',
    width=0.8  # Reduce the width of the bars
)

# Display plot with title, axis labels, and no legend
plot_display(
    title='Number of Appointments by Day of the Week',
    xlabel='Day of the Week',
    ylabel='Total Appointments',
    legendlabel= None
)

In [ ]:
# Utilisation rate only for weekdays
#Should the NHS start looking at increasing staff levels?
# Filter to include only rows where 'appointment_day_of_the_week' is not 'Saturday' or 'Sunday'
nc_weekdays = nc.query("appointment_day_of_the_week not in ['Saturday', 'Sunday']")

# Group by appointment_month to get total appointments per month for weekdays
nc_weekdays_appt = nc_weekdays.groupby('appointment_month_year', as_index=False)['count_of_appointments'].sum().rename(columns={'count_of_appointments': 'weekday_appointments'})

# Function to calculate the number of workdays in a given month and year
def count_workdays(year, month):
    # Get the number of days in the month
    num_days = calendar.monthrange(year, month)[1]
    # Count the number of workdays (Monday to Friday)
    workdays = sum(1 for day in range(1, num_days + 1) if calendar.weekday(year, month, day) < 5)
    return workdays

# Apply the function to each row to calculate workdays
nc_weekdays_appt['workdays'] = nc_weekdays_appt['appointment_month_year'].apply(lambda x: count_workdays(x.year, x.month))

# Add utilisation column (average daily appointments / NHS capacity)
# 1,200,000 is assumed NHS daily capacity, divide monthly total by the number of workdays
nc_weekdays_appt['utilisation'] = round((nc_weekdays_appt['weekday_appointments'] / nc_weekdays_appt['workdays']) / 1_200_000, 2)


# Monthly Appointments and Utilization Rate on Weekdays

## Analysis

- **Data Filtering**: Filtered to include only weekdays (Monday to Friday) to analyze appointment trends and utilization rates.
- **Data Grouping**: Grouped by appointment month to calculate the total number of appointments and workdays.
- **Utilization Rate Calculation**: Calculated the utilization rate as the ratio of average daily appointments to the assumed NHS daily capacity (1,200,000).

## Insights

### Monthly Appointments on Weekdays

- **Trends**:
  - The number of appointments on weekdays shows fluctuations over the observed period.
  - Peaks in appointment counts are observed around October 2021 and March 2022.
  - There are noticeable declines in appointment counts around December 2021 and May 2022.

### Monthly Utilization Rate on Weekdays

- **Utilization Rate Trends**:
  - The utilization rate fluctuates over time, with noticeable peaks and troughs.
  - The utilization rate is generally around or slightly above 1, indicating that the NHS capacity is being fully utilized or slightly overutilized during these periods.
  - Peaks in the utilization rate are observed around October 2021 and March 2022, aligning with the peaks in appointment counts.
  - Declines in the utilization rate are observed around December 2021 and May 2022, aligning with the declines in appointment counts.

## Conclusion

- **Staffing Considerations**:
  - The analysis suggests that the NHS is generally operating at or near full capacity on weekdays.
  - Given the fluctuations in appointment counts and utilization rates, the NHS may need to consider increasing staff levels during periods of high demand to maintain optimal service levels and reduce potential overutilization.

In [ ]:
# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))
#Plot monthly appointments on weekdays and utilization rate
sns.lineplot(
    data=nc_weekdays_appt,
    x='appointment_month_year',
    y='weekday_appointments',
    errorbar=None,
    marker='.'
)

# Display plot with title, axis labels, and legend
plot_display(
    title='Monthly Appointments on Weekdays',
    xlabel='Months',
    ylabel='Total Appointments',
    legendlabel=None,
    format_datetime=True
)

In [ ]:
# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))
#Plot monthly appointments on weekdays and utilization rate
axes_utilization = sns.lineplot(
    data=nc_weekdays_appt,
    x='appointment_month_year',
    y='utilisation',
    errorbar=None,
    marker='.'
)
# Add a horizontal dashed line at utilization rate of 1
plt.axhline(y=1, color='red', linestyle='--')
axes_utilization.set_ylim(0.8, 1.3)

# Display plot with title, axis labels, and legend
plot_display(
    title='Monthly Utilization Rate on Weekdays',
    xlabel='Months',
    ylabel='Utilization Rate',
    legendlabel=None,
    format_datetime=True
)

In [ ]:
# Group and sum 'fte_total' and 'employee_count' for GPs by month and ICB code
rd_gp_icb = rd[rd['staff_group'] == 'GP'].groupby(['appointment_month_year', 'icb_ons_code'])[['fte_total', 'employee_count']].sum().reset_index()

# Group and sum 'count_of_appointments' by month and ICB code
nc_gp_icb = nc_gp.groupby(['appointment_month_year', 'icb_ons_code'])['count_of_appointments'].sum().reset_index()

# Merge appointment counts with staff data
nc_rd_gp_icb = pd.merge(nc_gp_icb, rd_gp_icb, on=['appointment_month_year', 'icb_ons_code'], how='left')

# Calculate appointments per staff
nc_rd_gp_icb['appts_per_staff'] = nc_rd_gp_icb['count_of_appointments'] / nc_rd_gp_icb['fte_total']

# Convert 'appointment_month_year' to datetime and format as 'MMM-YYYY'
nc_rd_gp_icb['appointment_month_year'] = pd.to_datetime(nc_rd_gp_icb['appointment_month_year'])
nc_rd_gp_icb['appointment_month_year'] = nc_rd_gp_icb['appointment_month_year'].dt.strftime('%b-%Y')

# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Create a boxplot of appointments per staff by month
sns.boxplot(
    data=nc_rd_gp_icb,
    x='appointment_month_year',
    y='appts_per_staff',
    hue='appointment_month_year',
)

# Loop through each month to find and label outliers
months_gp_icb = nc_rd_gp_icb['appointment_month_year'].unique()

for month_iter_gp in months_gp_icb:
    month_data = nc_rd_gp_icb[nc_rd_gp_icb['appointment_month_year'] == month_iter_gp]
    q1 = month_data['appts_per_staff'].quantile(0.25)
    q3 = month_data['appts_per_staff'].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    outliers = month_data[(month_data['appts_per_staff'] < lower_bound) | (month_data['appts_per_staff'] > upper_bound)]

    for i, row_iter in outliers.iterrows():
        plt.text(
            x=month_iter_gp,
            y=row_iter['appts_per_staff'] * 0.971,
            s=row_iter['icb_ons_code'],
            fontsize=9,
            ha='center'
        )

# Display plot with title, axis labels, and legend
plot_display(
    title='Distribution of Appointments per GP by Month for all ICBs',
    xlabel='Months',
    ylabel='Appointments per Staff',
    legendlabel=None
)

In [ ]:
# Group and sum 'fte_total' and 'employee_count' for Nurses by month and ICB code
rd_nurse_icb = rd[rd['staff_group'] == 'Nurses'].groupby(['appointment_month_year', 'icb_ons_code'])[['fte_total', 'employee_count']].sum().reset_index()

# Group and sum 'count_of_appointments' by month and ICB code
nc_nurse_icb = nc_gp.groupby(['appointment_month_year', 'icb_ons_code'])['count_of_appointments'].sum().reset_index()

# Merge appointment counts with staff data
nc_rd_nurse_icb = pd.merge(nc_nurse_icb, rd_nurse_icb, on=['appointment_month_year', 'icb_ons_code'], how='left')

# Calculate appointments per staff
nc_rd_nurse_icb['appts_per_staff'] = nc_rd_nurse_icb['count_of_appointments'] / nc_rd_nurse_icb['fte_total']

# Convert 'appointment_month_year' to datetime and format as 'MMM-YYYY'
nc_rd_nurse_icb['appointment_month_year'] = pd.to_datetime(nc_rd_nurse_icb['appointment_month_year'])
nc_rd_nurse_icb['appointment_month_year'] = nc_rd_nurse_icb['appointment_month_year'].dt.strftime('%b-%Y')

# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Create a boxplot of appointments per staff by month
sns.boxplot(
    data=nc_rd_nurse_icb,
    x='appointment_month_year',
    y='appts_per_staff',
    hue='appointment_month_year',
)

# Loop through each month to find and label outliers
months_nurses_icb = nc_rd_nurse_icb['appointment_month_year'].unique()

for month_iter_nurses in months_nurses_icb:
    month_data = nc_rd_nurse_icb[nc_rd_nurse_icb['appointment_month_year'] == month_iter_nurses]
    q1 = month_data['appts_per_staff'].quantile(0.25)
    q3 = month_data['appts_per_staff'].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    outliers = month_data[(month_data['appts_per_staff'] < lower_bound) | (month_data['appts_per_staff'] > upper_bound)]

    for i, row_iter in outliers.iterrows():
        plt.text(
            x=month_iter_nurses,
            y=row_iter['appts_per_staff'] * 0.971,
            s=row_iter['icb_ons_code'],
            fontsize=9,
            ha='center'
        )

# Display plot with title, axis labels, and legend
plot_display(
    title='Distribution of Appointments per Nurse by Month for all ICBs',
    xlabel='Months',
    ylabel='Appointments per Staff',
    legendlabel=None
)

In [ ]:
# Group and sum 'fte_total' and 'employee_count' for DPCs by month and ICB code
rd_dpc_icb = rd[rd['staff_group'] == 'Direct Patient Care'].groupby(['appointment_month_year', 'icb_ons_code'])[['fte_total', 'employee_count']].sum().reset_index()

# Group and sum 'count_of_appointments' by month and ICB code
nc_dpc_icb = nc_gp.groupby(['appointment_month_year', 'icb_ons_code'])['count_of_appointments'].sum().reset_index()

# Merge appointment counts with staff data
nc_rd_dpc_icb = pd.merge(nc_dpc_icb, rd_dpc_icb, on=['appointment_month_year', 'icb_ons_code'], how='left')

# Calculate appointments per staff
nc_rd_dpc_icb['appts_per_staff'] = nc_rd_dpc_icb['count_of_appointments'] / nc_rd_dpc_icb['fte_total']

# Convert 'appointment_month_year' to datetime and format as 'MMM-YYYY'
nc_rd_dpc_icb['appointment_month_year'] = pd.to_datetime(nc_rd_dpc_icb['appointment_month_year'])
nc_rd_dpc_icb['appointment_month_year'] = nc_rd_dpc_icb['appointment_month_year'].dt.strftime('%b-%Y')

# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Create a boxplot of appointments per staff by month
sns.boxplot(
    data=nc_rd_dpc_icb,
    x='appointment_month_year',
    y='appts_per_staff',
    hue='appointment_month_year',
)

# Loop through each month to find and label outliers
months_dpc_icb = nc_rd_dpc_icb['appointment_month_year'].unique()

for month_iter_dpc in months_dpc_icb:
    month_data = nc_rd_dpc_icb[nc_rd_dpc_icb['appointment_month_year'] == month_iter_dpc]
    q1 = month_data['appts_per_staff'].quantile(0.25)
    q3 = month_data['appts_per_staff'].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    outliers = month_data[(month_data['appts_per_staff'] < lower_bound) | (month_data['appts_per_staff'] > upper_bound)]

    for i, row_iter in outliers.iterrows():
        plt.text(
            x=month_iter_dpc,
            y=row_iter['appts_per_staff'] * 0.971,
            s=row_iter['icb_ons_code'],
            fontsize=9,
            ha='center'
        )

# Display plot with title, axis labels, and legend
plot_display(
    title='Distribution of Appointments per DPC by Month for all ICBs',
    xlabel='Months',
    ylabel='Appointments per Staff',
    legendlabel=None
)

In [ ]:
# Group and sum 'fte_total' and 'employee_count' for Admin by month and ICB code
rd_admin_icb = rd[rd['staff_group'] == 'Admin/Non-Clinical'].groupby(['appointment_month_year', 'icb_ons_code'])[['fte_total', 'employee_count']].sum().reset_index()

# Group and sum 'count_of_appointments' by month and ICB code
nc_admin_icb = nc_gp.groupby(['appointment_month_year', 'icb_ons_code'])['count_of_appointments'].sum().reset_index()

# Merge appointment counts with staff data
nc_rd_admin_icb = pd.merge(nc_admin_icb, rd_admin_icb, on=['appointment_month_year', 'icb_ons_code'], how='left')

# Calculate appointments per staff
nc_rd_admin_icb['appts_per_staff'] = nc_rd_admin_icb['count_of_appointments'] / nc_rd_admin_icb['fte_total']

# Convert 'appointment_month_year' to datetime and format as 'MMM-YYYY'
nc_rd_admin_icb['appointment_month_year'] = pd.to_datetime(nc_rd_admin_icb['appointment_month_year'])
nc_rd_admin_icb['appointment_month_year'] = nc_rd_admin_icb['appointment_month_year'].dt.strftime('%b-%Y')

# Initialize plot
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Create a boxplot of appointments per staff by month
sns.boxplot(
    data=nc_rd_admin_icb,
    x='appointment_month_year',
    y='appts_per_staff',
    hue='appointment_month_year',
)

# Loop through each month to find and label outliers
months_admin_icb = nc_rd_admin_icb['appointment_month_year'].unique()

for months_iter_admin in months_admin_icb:
    month_data = nc_rd_admin_icb[nc_rd_admin_icb['appointment_month_year'] == months_iter_admin]
    q1 = month_data['appts_per_staff'].quantile(0.25)
    q3 = month_data['appts_per_staff'].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    outliers = month_data[(month_data['appts_per_staff'] < lower_bound) | (month_data['appts_per_staff'] > upper_bound)]

    for i, row_iter in outliers.iterrows():
        plt.text(
            x=months_iter_admin,
            y=row_iter['appts_per_staff'] * 0.971,
            s=row_iter['icb_ons_code'],
            fontsize=9,
            ha='center'
        )

# Display plot with title, axis labels, and legend
plot_display(
    title='Distribution of Appointments per Admin by Month for all ICBs',
    xlabel='Months',
    ylabel='Appointments per Staff',
    legendlabel=None
)

In [ ]:
# Define a function to remove outliers based on IQR per group
def remove_outliers_iqr(df, group_col, value_col):
    cleaned_df = pd.DataFrame()

    for name, group_data in df.groupby(group_col):
        q1_data = group_data[value_col].quantile(0.25)
        q3_data = group_data[value_col].quantile(0.75)
        iqr_data = q3_data - q1_data
        lower_bound_data = q1_data - 1.5 * iqr_data
        upper_bound_data = q3_data + 1.5 * iqr_data

        # Filter to remove outliers
        group_no_outliers = group_data[(group_data[value_col] >= lower_bound_data) & (group_data[value_col] <= upper_bound_data)]
        cleaned_df = pd.concat([cleaned_df, group_no_outliers], axis=0)

    return cleaned_df



In [ ]:
# Apply the function
nc_rd_gp_icb_no_outliers = remove_outliers_iqr(nc_rd_gp_icb, group_col='appointment_month_year', value_col='appts_per_staff')
nc_rd_nurse_icb_no_outliers = remove_outliers_iqr(nc_rd_nurse_icb, group_col='appointment_month_year', value_col='appts_per_staff')
nc_rd_dpc_icb_no_outliers = remove_outliers_iqr(nc_rd_dpc_icb, group_col='appointment_month_year', value_col='appts_per_staff')
nc_rd_admin_icb_no_outliers = remove_outliers_iqr(nc_rd_admin_icb, group_col='appointment_month_year', value_col='appts_per_staff')

staff_types_no_outliers = [
    ('GP', nc_rd_gp_icb_no_outliers),
    ('Nurses', nc_rd_nurse_icb_no_outliers),
    ('DPC', nc_rd_dpc_icb_no_outliers),
    ('Admin/Non-Clinical', nc_rd_admin_icb_no_outliers)
]

# Initialize plot with specified figure size for staff type trends
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Plot appointments per staff per day for each staff type
for label, rd_group in staff_types_no_outliers:
    rd_group = rd_group.groupby(rd_group['appointment_month_year'])['appts_per_staff'].mean().reset_index()
    rd_group['appts_per_staff_per_day'] = rd_group['appts_per_staff']/30
    # Convert 'appointment_month_year' to datetime and format as 'MMM-YYYY'
    rd_group['appointment_month_year'] = pd.to_datetime(rd_group['appointment_month_year'])
    rd_group['appointment_month_year'] = rd_group['appointment_month_year'].dt.strftime('%b-%Y')
    rd_group['appointment_month_year'] = pd.to_datetime(rd_group['appointment_month_year'])

    sns.lineplot(
        data=rd_group,
        x='appointment_month_year',
        y='appts_per_staff_per_day',
        marker=marker_map_rd[label],
        color= color_map_rd[label],
        label=label,
        errorbar=None
    )

# Display plot with title, axis labels, and legend
plot_display(
    title='Daily average Appointments per Staff Type without Outliers ',
    xlabel='Appointment Month-Year',
    ylabel='Appointments per FTE',
    legendlabel='Staff Group',
    format_datetime = True
)

Based on this graph deep diving into ICBs that have the highest cancelled/missed appointments.\
ar - time between book and appt, appointment mode, ad - average duration of appointments for those ICBs, nc - context type and national category. Then compare with general to analyse.

E54000057 - 837503\
E54000008 - 732856\
E54000050 - 728088\
E54000027 - 587323\
E54000029 - 584327

Highest by ratio:\
E54000029 - 6.093791242386633\
E54000062 - 6.059937412180924\
E54000057 - 6.043505870297809\
E54000055 - 5.9759113222863185\
E54000030 - 5.709239417745094

In both there is an overlap of 2 ICBs - E54000057, E54000029. We will explore those in further detail to understand the reasons.


In [ ]:
# Filter ar_time_aft082021 for rows where icb_ons_code is 'E54000057'
ar_icb57 = ar_time_aft082021[ar_time_aft082021['icb_ons_code'] == 'E54000057']

# Filter ar_dna_time_aft082021 for rows where icb_ons_code is 'E54000057'
dna_icb57 = ar_dna_time_aft082021[ar_dna_time_aft082021['icb_ons_code'] == 'E54000057']



In [ ]:
# Time between book and appointment for total and cancelled appointments for ICB E54000057.
ar_between_icb57 = ar_icb57.groupby(['time_between_book_and_appointment'])['count_of_appointments'].sum().reset_index()
ar_dna_between_icb57 = dna_icb57.groupby(['time_between_book_and_appointment'])['count_of_appointments'].sum().reset_index()

ar_dna_icb57_time = pd.merge(ar_between_icb57, ar_dna_between_icb57, on='time_between_book_and_appointment', how='inner')
ar_dna_icb57_time['percentage_of_missed_appointments'] = ar_dna_icb57_time['count_of_appointments_y']/ar_dna_icb57_time['count_of_appointments_x']

# Create a figure with two subplots side by side
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

# First barplot: ar_icb57
sns.barplot(
    data=ar_dna_icb57_time,
    x='time_between_book_and_appointment',
    y='percentage_of_missed_appointments',
    order=time_between_order,
    errorbar=None,
    palette='viridis',
    ax=axes[0]  # Use the first subplot
)
# Second barplot: dna_icb57
sns.barplot(
    data=ar_dna_between_icb57,
    x='time_between_book_and_appointment',
    y='count_of_appointments',
    order=time_between_order,
    errorbar=None,
    palette='viridis',
    ax=axes[1]  # Use the second subplot
)

# Display the first plot with title and axis labels using plot_display
plot_display(
    title='Ratio missed appointments (ICB E54000057) by Time Between Book and Appointments',
    xlabel='Time Between Booking and Appointment',
    ylabel='% Missed Appointments',
    legendlabel=None,
    ax=axes[0]  # Pass the first subplot axis to plot_display
)
# Display the second plot with title and axis labels using plot_display
plot_display(
    title='Missed Appointments (ICB E54000057) by Time Between Book and Appointments',
    xlabel='Time Between Booking and Appointment',
    ylabel='Missed Appointments',
    legendlabel=None,
    ax=axes[1]  # Pass the second subplot axis to plot_display
)

plt.tight_layout()
plt.show()

In [ ]:
# Time between book and appointment for total and cancelled appointments for ICB E54000057.
ar_mode_icb57 = ar_icb57.groupby(['appointment_mode'])['count_of_appointments'].sum().reset_index()
ar_dna_mode_icb57 = dna_icb57.groupby(['appointment_mode'])['count_of_appointments'].sum().reset_index()

ar_dna_icb57_mode = pd.merge(ar_mode_icb57, ar_dna_mode_icb57, on='appointment_mode', how='inner')
ar_dna_icb57_mode['percentage_of_missed_appointments'] = ar_dna_icb57_mode['count_of_appointments_y']/ar_dna_icb57_mode['count_of_appointments_x']

# Create a figure with two subplots side by side
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

# First barplot: ar_icb57
sns.barplot(
    data=ar_dna_icb57_mode,
    x='appointment_mode',
    y='percentage_of_missed_appointments',
    errorbar=None,
    palette='viridis',
    ax=axes[0]  # Use the first subplot
)
# Second barplot: dna_icb57
sns.barplot(
    data=ar_dna_mode_icb57,
    x='appointment_mode',
    y='count_of_appointments',
    errorbar=None,
    palette='viridis',
    ax=axes[1]  # Use the second subplot
)

# Display the first plot with title and axis labels using plot_display
plot_display(
    title='Ratio Missed Appointments (ICB E54000057) by Mode',
    xlabel='Appointment Mode',
    ylabel='% Missed Appointments',
    legendlabel=None,
    ax=axes[0]  # Pass the first subplot axis to plot_display
)
# Display the second plot with title and axis labels using plot_display
plot_display(
    title='Missed Appointments (ICB E54000057) by Mode',
    xlabel='Appointment Mode',
    ylabel='Missed Appointments',
    legendlabel=None,
    ax=axes[1]  # Pass the second subplot axis to plot_display
)

plt.tight_layout()
plt.show()

In [ ]:
nc_icb57 = nc[nc['icb_ons_code'] == 'E54000057']
nc_ss_icb57 = nc_icb57.groupby(['appointment_month_year', 'service_setting'], as_index=False)['count_of_appointments'].sum()

# Filter out rows where 'actual_duration' is 'Unknown / Data Quality'
filtered_nc_icb57 = nc_ss_icb57[nc_ss_icb57['service_setting'] != 'Unknown / Data Quality']

# Group by 'actual_duration' and sum 'count_of_appointments'
nc_icb57_group = filtered_nc_icb57.groupby('service_setting', as_index=False)['count_of_appointments'].sum()

# First barplot: ar_icb57
sns.barplot(
    data=nc_icb57_group,
    x='service_setting',
    y='count_of_appointments',
    errorbar=None,
    palette='viridis',
)

# Display plot with title, axis labels, and no legend
plot_display(
    title='Appointments by service setting (ICB E54000057)',
    xlabel='Service Setting',
    ylabel='Total Appointments',
    legendlabel= None
)

In [ ]:
# ICB E54000057
rd_icb57 = rd[rd['icb_ons_code'] == 'E54000057']

nc_icb57_group = nc_icb57.groupby(['appointment_month_year'])['count_of_appointments'].sum().reset_index()
rd_icb57_group = rd_icb57.groupby('appointment_month_year')[['fte_total', 'employee_count']].sum().reset_index()

# Splitting each staff group into a separate database to further evaluate each staff group separately and advise accordingly where we would need to add resources if needed.
rd_gp_icb57 = rd_icb57[rd_icb57['staff_group'] == 'GP'].groupby('appointment_month_year')[['fte_total', 'employee_count']].sum().reset_index()
rd_nurse_icb57 = rd_icb57[rd_icb57['staff_group'] == 'Nurses'].groupby('appointment_month_year')[['fte_total', 'employee_count']].sum().reset_index()
rd_dpc_icb57 = rd_icb57[rd_icb57['staff_group'] == 'Direct Patient Care'].groupby('appointment_month_year')[['fte_total', 'employee_count']].sum().reset_index()
rd_admin_icb57 = rd_icb57[rd_icb57['staff_group'] == 'Admin/Non-Clinical'].groupby('appointment_month_year')[['fte_total', 'employee_count']].sum().reset_index()


# Merge data and calculate appointments per staff per day for each staff type
staff_types_icb57 = [
    ('GP', nc_icb57_group, rd_gp_icb57),
    ('Nurses', nc_icb57_group, rd_nurse_icb57),
    ('DPC', nc_icb57_group, rd_dpc_icb57),
    ('Admin/Non-Clinical', nc_icb57_group, rd_admin_icb57)
]

# Initialize plot with specified figure size for staff type trends
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Plot appointments per staff per day for each staff type
for label, nc_group, rd_group in staff_types_icb57:
    nc_rd_staff = pd.merge(nc_group, rd_group, on=['appointment_month_year'], how='left')
    nc_rd_staff['appts_per_staff'] = nc_rd_staff['count_of_appointments'] / nc_rd_staff['fte_total']
    nc_rd_staff['appts_per_staff_per_day'] = nc_rd_staff['appts_per_staff'] / 30

    sns.lineplot(
        data=nc_rd_staff,
        x='appointment_month_year',
        y='appts_per_staff_per_day',
        marker=marker_map_rd[label],
        color= color_map_rd[label],
        label=label,
        errorbar=None
    )

# Display plot with title, axis labels, and legend
plot_display(
    title='Daily Appointments per Staff Type for E54000057 ',
    xlabel='Appointment Month-Year',
    ylabel='Appointments per FTE',
    legendlabel='Staff Group',
    format_datetime=True
)


In [ ]:
# Filter ar_time_aft082021 for rows where icb_ons_code is 'E54000029'
ar_icb29 = ar_time_aft082021[ar_time_aft082021['icb_ons_code'] == 'E54000029']

# Filter ar_dna_time_aft082021 for rows where icb_ons_code is 'E54000029'
dna_icb29 = ar_dna_time_aft082021[ar_dna_time_aft082021['icb_ons_code'] == 'E54000029']



In [ ]:
# Time between book and appointment for total and cancelled appointments for ICB E54000029.
ar_between_icb29 = ar_icb29.groupby(['time_between_book_and_appointment'])['count_of_appointments'].sum().reset_index()
ar_dna_between_icb29 = dna_icb29.groupby(['time_between_book_and_appointment'])['count_of_appointments'].sum().reset_index()

ar_dna_icb29_time = pd.merge(ar_between_icb29, ar_dna_between_icb29, on='time_between_book_and_appointment', how='inner')
ar_dna_icb29_time['percentage_of_missed_appointments'] = ar_dna_icb29_time['count_of_appointments_y']/ar_dna_icb29_time['count_of_appointments_x']

# Create a figure with two subplots side by side
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

# First barplot: ar_icb29
sns.barplot(
    data=ar_dna_icb29_time,
    x='time_between_book_and_appointment',
    y='percentage_of_missed_appointments',
    order=time_between_order,
    errorbar=None,
    palette='viridis',
    ax=axes[0]  # Use the first subplot
)
# Second barplot: dna_icb29
sns.barplot(
    data=ar_dna_between_icb29,
    x='time_between_book_and_appointment',
    y='count_of_appointments',
    order=time_between_order,
    errorbar=None,
    palette='viridis',
    ax=axes[1]  # Use the second subplot
)

# Display the first plot with title and axis labels using plot_display
plot_display(
    title='Ratio missed appointments (ICB E54000029) by Time Between Book and Appointments',
    xlabel='Time Between Booking and Appointment',
    ylabel='% Missed Appointments',
    legendlabel=None,
    ax=axes[0]  # Pass the first subplot axis to plot_display
)
# Display the second plot with title and axis labels using plot_display
plot_display(
    title='Missed Appointments (ICB E54000029) by Time Between Book and Appointments',
    xlabel='Time Between Booking and Appointment',
    ylabel='Missed Appointments',
    legendlabel=None,
    ax=axes[1]  # Pass the second subplot axis to plot_display
)

plt.tight_layout()
plt.show()

In [ ]:
# Time between book and appointment for total and cancelled appointments for ICB E54000029.
ar_mode_icb29 = ar_icb29.groupby(['appointment_mode'])['count_of_appointments'].sum().reset_index()
ar_dna_mode_icb29 = dna_icb29.groupby(['appointment_mode'])['count_of_appointments'].sum().reset_index()

ar_dna_icb29_mode = pd.merge(ar_mode_icb29, ar_dna_mode_icb29, on='appointment_mode', how='inner')
ar_dna_icb29_mode['percentage_of_missed_appointments'] = ar_dna_icb29_mode['count_of_appointments_y']/ar_dna_icb29_mode['count_of_appointments_x']

# Create a figure with two subplots side by side
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

# First barplot: ar_icb29
sns.barplot(
    data=ar_dna_icb29_mode,
    x='appointment_mode',
    y='percentage_of_missed_appointments',
    errorbar=None,
    palette='viridis',
    ax=axes[0]  # Use the first subplot
)
# Second barplot: dna_icb29
sns.barplot(
    data=ar_dna_mode_icb29,
    x='appointment_mode',
    y='count_of_appointments',
    errorbar=None,
    palette='viridis',
    ax=axes[1]  # Use the second subplot
)

# Display the first plot with title and axis labels using plot_display
plot_display(
    title='Ratio Missed Appointments (ICB E54000029) by Mode',
    xlabel='Appointment Mode',
    ylabel='% Missed Appointments',
    legendlabel=None,
    ax=axes[0]  # Pass the first subplot axis to plot_display
)
# Display the second plot with title and axis labels using plot_display
plot_display(
    title='Missed Appointments (ICB E54000029) by Mode',
    xlabel='Appointment Mode',
    ylabel='Missed Appointments',
    legendlabel=None,
    ax=axes[1]  # Pass the second subplot axis to plot_display
)

plt.tight_layout()
plt.show()

In [ ]:
nc_icb29 = nc[nc['icb_ons_code'] == 'E54000029']
nc_ss_icb29 = nc_icb29.groupby(['appointment_month_year', 'service_setting'], as_index=False)['count_of_appointments'].sum()

# Filter out rows where 'actual_duration' is 'Unknown / Data Quality'
filtered_nc_icb29 = nc_ss_icb29[nc_ss_icb29['service_setting'] != 'Unknown / Data Quality']

# Group by 'actual_duration' and sum 'count_of_appointments'
nc_icb29_group = filtered_nc_icb29.groupby('service_setting', as_index=False)['count_of_appointments'].sum()

# First barplot: ar_icb29
sns.barplot(
    data=nc_icb29_group,
    x='service_setting',
    y='count_of_appointments',
    errorbar=None,
    palette='viridis',
)

# Display plot with title, axis labels, and no legend
plot_display(
    title='Appointments by service setting (ICB E54000029)',
    xlabel='Service Setting',
    ylabel='Total Appointments',
    legendlabel= None
)

In [ ]:
# ICB E54000029
rd_icb29 = rd[rd['icb_ons_code'] == 'E54000029']

nc_icb29_group = nc_icb29.groupby(['appointment_month_year'])['count_of_appointments'].sum().reset_index()
rd_icb29_group = rd_icb29.groupby('appointment_month_year')[['fte_total', 'employee_count']].sum().reset_index()

# Splitting each staff group into a separate database to further evaluate each staff group separately and advise accordingly where we would need to add resources if needed.
rd_gp_icb29 = rd_icb29[rd_icb29['staff_group'] == 'GP'].groupby('appointment_month_year')[['fte_total', 'employee_count']].sum().reset_index()
rd_nurse_icb29 = rd_icb29[rd_icb29['staff_group'] == 'Nurses'].groupby('appointment_month_year')[['fte_total', 'employee_count']].sum().reset_index()
rd_dpc_icb29 = rd_icb29[rd_icb29['staff_group'] == 'Direct Patient Care'].groupby('appointment_month_year')[['fte_total', 'employee_count']].sum().reset_index()
rd_admin_icb29 = rd_icb29[rd_icb29['staff_group'] == 'Admin/Non-Clinical'].groupby('appointment_month_year')[['fte_total', 'employee_count']].sum().reset_index()


# Merge data and calculate appointments per staff per day for each staff type
staff_types_icb29 = [
    ('GP', nc_icb29_group, rd_gp_icb29),
    ('Nurses', nc_icb29_group, rd_nurse_icb29),
    ('DPC', nc_icb29_group, rd_dpc_icb29),
    ('Admin/Non-Clinical', nc_icb29_group, rd_admin_icb29)
]

# Initialize plot with specified figure size for staff type trends
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Plot appointments per staff per day for each staff type
for label, nc_group, rd_group in staff_types_icb29:
    nc_rd_staff = pd.merge(nc_group, rd_group, on=['appointment_month_year'], how='left')
    nc_rd_staff['appts_per_staff'] = nc_rd_staff['count_of_appointments'] / nc_rd_staff['fte_total']
    nc_rd_staff['appts_per_staff_per_day'] = nc_rd_staff['appts_per_staff'] / 30

    sns.lineplot(
        data=nc_rd_staff,
        x='appointment_month_year',
        y='appts_per_staff_per_day',
        marker=marker_map_rd[label],
        color= color_map_rd[label],
        label=label,
        errorbar=None
    )

# Display plot with title, axis labels, and legend
plot_display(
    title='Daily Appointments per Staff Type for E54000029 ',
    xlabel='Appointment Month-Year',
    ylabel='Appointments per FTE',
    legendlabel='Staff Group',
    format_datetime=True
)


In [ ]:
# Filter ar_time_aft082021 for rows where icb_ons_code is 'E54000062_55_30'
ar_icb62_55_30 = ar_time_aft082021[ar_time_aft082021['icb_ons_code'].isin(['E54000062', 'E54000055', 'E54000030'])]

# Filter ar_dna_time_aft082021 for rows where icb_ons_code is 'E54000062_55_30'
dna_icb62_55_30 = ar_dna_time_aft082021[ar_dna_time_aft082021['icb_ons_code'].isin(['E54000062', 'E54000055', 'E54000030'])]

In [ ]:
# Group data by month and national category to sum appointments
nc_icb62_55_30 = nc_gp[nc_gp['icb_ons_code'].isin(['E54000062', 'E54000055', 'E54000030'])]

In [ ]:
# Time between book and appointment for total and cancelled appointments for ICB E54000062_55_30.
ar_between_icb62_55_30 = ar_icb62_55_30.groupby(['time_between_book_and_appointment'])['count_of_appointments'].sum().reset_index()
ar_dna_between_icb62_55_30 = dna_icb62_55_30.groupby(['time_between_book_and_appointment'])['count_of_appointments'].sum().reset_index()

ar_dna_icb62_55_30_time = pd.merge(ar_between_icb62_55_30, ar_dna_between_icb62_55_30, on='time_between_book_and_appointment', how='inner')
ar_dna_icb62_55_30_time['percentage_of_missed_appointments'] = ar_dna_icb62_55_30_time['count_of_appointments_y']/ar_dna_icb62_55_30_time['count_of_appointments_x']

# Create a figure with two subplots side by side
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

# First barplot: ar_icb62_55_30
sns.barplot(
    data=ar_dna_icb62_55_30_time,
    x='time_between_book_and_appointment',
    y='percentage_of_missed_appointments',
    order=time_between_order,
    errorbar=None,
    palette='viridis',
    ax=axes[0]  # Use the first subplot
)
# Second barplot: dna_icb62_55_30
sns.barplot(
    data=ar_dna_between_icb62_55_30,
    x='time_between_book_and_appointment',
    y='count_of_appointments',
    order=time_between_order,
    errorbar=None,
    palette='viridis',
    ax=axes[1]  # Use the second subplot
)

# Display the first plot with title and axis labels using plot_display
plot_display(
    title='Ratio missed appointments (ICB E54000062_55_30) by Time Between Book and Appointments',
    xlabel='Time Between Booking and Appointment',
    ylabel='% Missed Appointments',
    legendlabel=None,
    ax=axes[0]  # Pass the first subplot axis to plot_display
)
# Display the second plot with title and axis labels using plot_display
plot_display(
    title='Missed Appointments (ICB E54000062_55_30) by Time Between Book and Appointments',
    xlabel='Time Between Booking and Appointment',
    ylabel='Missed Appointments',
    legendlabel=None,
    ax=axes[1]  # Pass the second subplot axis to plot_display
)

plt.tight_layout()
plt.show()

In [ ]:
 # Time between book and appointment for total and cancelled appointments for ICB E54000062_55_30.
ar_mode_icb62_55_30 = ar_icb62_55_30.groupby(['appointment_mode'])['count_of_appointments'].sum().reset_index()
ar_dna_mode_icb62_55_30 = dna_icb62_55_30.groupby(['appointment_mode'])['count_of_appointments'].sum().reset_index()

ar_dna_icb62_55_30_mode = pd.merge(ar_mode_icb62_55_30, ar_dna_mode_icb62_55_30, on='appointment_mode', how='inner')
ar_dna_icb62_55_30_mode['percentage_of_missed_appointments'] = ar_dna_icb62_55_30_mode['count_of_appointments_y']/ar_dna_icb62_55_30_mode['count_of_appointments_x']

# Create a figure with two subplots side by side
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

# First barplot: ar_icb62_55_30
sns.barplot(
    data=ar_dna_icb62_55_30_mode,
    x='appointment_mode',
    y='percentage_of_missed_appointments',
    errorbar=None,
    palette='viridis',
    ax=axes[0]  # Use the first subplot
)
# Second barplot: dna_icb62_55_30
sns.barplot(
    data=ar_dna_mode_icb62_55_30,
    x='appointment_mode',
    y='count_of_appointments',
    errorbar=None,
    palette='viridis',
    ax=axes[1]  # Use the second subplot
)

# Display the first plot with title and axis labels using plot_display
plot_display(
    title='Ratio Missed Appointments (ICB E54000062_55_30) by Mode',
    xlabel='Appointment Mode',
    ylabel='% Missed Appointments',
    legendlabel=None,
    ax=axes[0]  # Pass the first subplot axis to plot_display
)
# Display the second plot with title and axis labels using plot_display
plot_display(
    title='Missed Appointments (ICB E54000062_55_30) by Mode',
    xlabel='Appointment Mode',
    ylabel='Missed Appointments',
    legendlabel=None,
    ax=axes[1]  # Pass the second subplot axis to plot_display
)

plt.tight_layout()
plt.show()

In [ ]:

nc_ss_icb62_55_30 = nc_icb62_55_30.groupby(['appointment_month_year', 'service_setting'], as_index=False)['count_of_appointments'].sum()

# Filter out rows where 'actual_duration' is 'Unknown / Data Quality'
filtered_nc_icb62_55_30 = nc_ss_icb62_55_30[nc_ss_icb62_55_30['service_setting'] != 'Unknown / Data Quality']

# Group by 'actual_duration' and sum 'count_of_appointments'
nc_icb62_55_30_group = filtered_nc_icb62_55_30.groupby('service_setting', as_index=False)['count_of_appointments'].sum()

# First barplot: ar_icb62_55_30
sns.barplot(
    data=nc_icb62_55_30_group,
    x='service_setting',
    y='count_of_appointments',
    errorbar=None,
    palette='viridis',
)

# Display plot with title, axis labels, and no legend
plot_display(
    title='Appointments by service setting (ICB E54000062_55_30)',
    xlabel='Service Setting',
    ylabel='Total Appointments',
    legendlabel= None
)

In [ ]:
# ICB E54000062_55_30
rd_icb62_55_30 = rd[rd['icb_ons_code'].isin(['E54000062', 'E54000055', 'E54000030'])]

nc_icb62_55_30_group = nc_icb62_55_30.groupby(['appointment_month_year'])['count_of_appointments'].sum().reset_index()
rd_icb62_55_30_group = rd_icb62_55_30.groupby('appointment_month_year')[['fte_total', 'employee_count']].sum().reset_index()

# Splitting each staff group into a separate database to further evaluate each staff group separately and advise accordingly where we would need to add resources if needed.
rd_gp_icb62_55_30 = rd_icb62_55_30[rd_icb62_55_30['staff_group'] == 'GP'].groupby('appointment_month_year')[['fte_total', 'employee_count']].sum().reset_index()
rd_nurse_icb62_55_30 = rd_icb62_55_30[rd_icb62_55_30['staff_group'] == 'Nurses'].groupby('appointment_month_year')[['fte_total', 'employee_count']].sum().reset_index()
rd_dpc_icb62_55_30 = rd_icb62_55_30[rd_icb62_55_30['staff_group'] == 'Direct Patient Care'].groupby('appointment_month_year')[['fte_total', 'employee_count']].sum().reset_index()
rd_admin_icb62_55_30 = rd_icb62_55_30[rd_icb62_55_30['staff_group'] == 'Admin/Non-Clinical'].groupby('appointment_month_year')[['fte_total', 'employee_count']].sum().reset_index()


# Merge data and calculate appointments per staff per day for each staff type
staff_types_icb62_55_30 = [
    ('GP', nc_icb62_55_30_group, rd_gp_icb62_55_30),
    ('Nurses', nc_icb62_55_30_group, rd_nurse_icb62_55_30),
    ('DPC', nc_icb62_55_30_group, rd_dpc_icb62_55_30),
    ('Admin/Non-Clinical', nc_icb62_55_30_group, rd_admin_icb62_55_30)
]

# Initialize plot with specified figure size for staff type trends
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Plot appointments per staff per day for each staff type
for label, nc_group, rd_group in staff_types_icb62_55_30:
    nc_rd_staff = pd.merge(nc_group, rd_group, on=['appointment_month_year'], how='left')
    nc_rd_staff['appts_per_staff'] = nc_rd_staff['count_of_appointments'] / nc_rd_staff['fte_total']
    nc_rd_staff['appts_per_staff_per_day'] = nc_rd_staff['appts_per_staff'] / 30

    sns.lineplot(
        data=nc_rd_staff,
        x='appointment_month_year',
        y='appts_per_staff_per_day',
        marker=marker_map_rd[label],
        color= color_map_rd[label],
        label=label,
        errorbar=None
    )

# Display plot with title, axis labels, and legend
plot_display(
    title='Daily Appointments per Staff Type for E54000062_55_30 ',
    xlabel='Appointment Month-Year',
    ylabel='Appointments per FTE',
    legendlabel='Staff Group',
    format_datetime=True
)


# Data Cleaning Process

## Overview

- **Data Loading**: The script loads data from a CSV file and converts it to a pickle file for faster loading in subsequent runs.
- **Data Cleaning**: The script performs several data cleaning steps to ensure the dataset is ready for analysis.

## Data Cleaning Steps

1. **Handling Missing Values**:
   - Identify and remove rows with missing values from the DataFrame.

2. **Removing Duplicates**:
   - Identify and remove duplicate rows from the DataFrame, keeping the first occurrence.
   - Reset the index of the DataFrame after dropping rows.

3. **Text Cleaning**:
   - **Removing Hyperlinks**: Create a new column `clean_text` by removing hyperlinks from the `tweet_full_text` column.
   - **Removing Emojis**: Define and apply a function to remove emojis from the `clean_text` column.
   - **Removing Punctuation**: Remove punctuation from the `clean_text` column.
   - **Removing Mentions**: Remove mentions (words starting with @) from the `clean_text` column.

4. **Extracting Clean Text**:
   - Extract the `clean_text` column into a separate variable and convert it to a list.



In [ ]:
%%skip_if no_cleaning
# This cell will be skipped if no_cleaning is True

# If pickle file exists, read it; otherwise, read the CSV and convert to pickle
# This ensures faster loading from the second run and avoids reading the CSV file when resetting the kernel
if not os.path.exists("tweets.pickle"):
    # Read the CSV file and convert it to a pickle file
    tweets = pd.read_csv('tweets.csv')
    tweets.to_pickle('tweets.pickle')
    # Read the newly created pickle file
    tweets = pd.read_pickle('tweets.pickle')
else:
    # Read the existing pickle file
    tweets = pd.read_pickle('tweets.pickle')

# View the DataFrame (commented out for execution)
# print("Head \n", tweets.head())
# print("Types\n", tweets.dtypes)
# print("Shape\n", tweets.shape)
# print("Describe \n", tweets.describe)

In [ ]:
%%skip_if no_cleaning
# This cell will be skipped if no_cleaning is True

# Data Cleaning
# Determine whether there are missing values in the DataFrame
tweets_na = tweets[tweets.isna().any(axis=1)]

# Drop rows with missing values from the DataFrame
tweets.drop(tweets_na.index, axis=0, inplace=True)

# Identify duplicate rows in the DataFrame

tweets = tweets.drop_duplicates(subset='tweet_full_text', keep="first")
tweets = tweets.reset_index(drop=True)



In [ ]:

# Reset the index of the DataFrame after dropping duplicates
tweets = tweets.reset_index(drop=True)

# Create a new column 'clean_text' by removing hyperlinks from 'tweet_full_text'
tweets["clean_text"] = tweets["tweet_full_text"].apply(
    lambda x: re.sub('http[s]?://\S+', '', str(x)))

# Define a function to remove emojis from a string
def de_emojify(input_string):
    return input_string.encode('ascii', 'ignore').decode('ascii')

# Apply the deEmojify function to remove emojis from the 'clean_text' column
tweets["clean_text"] = tweets["clean_text"].apply(lambda s: de_emojify(s))

# Remove punctuation from the 'clean_text' column
tweets["clean_text"] = tweets["clean_text"].apply(
    lambda s: ' '.join(re.sub("[.,!?:;-='_]", " ", s).split()))

# Remove mentions (words starting with @) from the 'clean_text' column
tweets["clean_text"] = tweets["clean_text"].apply(
    lambda s: ' '.join(re.sub("@[A-Za-z0-9_]+", "", s).split()))

# Extract the 'clean_text' column into a separate variable
tweets_clean = tweets["clean_text"]

# Convert the 'clean_text' column to a list
tweets_clean = tweets_clean.tolist()


In [ ]:
# Create a list of values containing the # symbol.
tags = []
for y in [x.split(' ') for x in tweets['tweet_full_text'].values]:
    for z in y:
        if '#' in z:
            # Change to lowercase.
            tags.append(z.lower())

In [ ]:
# Convert the series to a DataFrame in preparation for visualisation.
tags = pd.Series(tags).value_counts()


In [ ]:
tags = pd.DataFrame(tags).reset_index()

# Rename the columns.
tags.columns = ['hashtag', 'count']
tags.head(30)

In [ ]:
# Filter hashtags with count > 10
filtered_tags = tags[tags['count'] > 10]

# Initialize plot with specified figure size for staff type trends
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Plot
sns.barplot(
    data=filtered_tags,
    x='hashtag',
    y='count',
    errorbar=None,
    palette='viridis'
)

# Display plot with title and axis labels
plot_display(
    title='Highest Number of Hashtags',
    xlabel='Hashtag',
    ylabel='Count of Tweets',
    legendlabel=None,
    xticksrotation = 90
)

In [ ]:
# Create a Seaborn barplot indicating records with a count <80 records to examine without the outlier .
filtered_tags2 = tags[(tags['count'] > 10) & (tags['count'] < 80)]

# Initialize plot with specified figure size for staff type trends
plt.figure(figsize=(standard_plot_width, standard_plot_height))

# Plot
sns.barplot(
    data=filtered_tags2,
    x='hashtag',
    y='count',
    errorbar=None,
    palette='viridis'
)

# Display plot with title and axis labels
plot_display(
    title='Highest Number of Hashtags without #Healthcare',
    xlabel='Hashtag',
    ylabel='Count of Tweets',
    legendlabel=None,
    xticksrotation = 90
)


# Counting Occurrences of Search Terms in Tweets

## Overview

- **Objective**: Count occurrences of specific search terms within a list of cleaned tweets.
- **Search Terms**: 'NHS', 'hiring', 'UK', 'US', 'AI'.

## Results

- Number of tweets containing 'NHS': 1
- Number of tweets containing 'hiring': 44
- Number of tweets containing 'UK': 8
- Number of tweets containing 'US': 274
- Number of tweets containing 'AI': 184

## Conclusion

Given the low number of tweets containing relevant keywords, this data will not be used for further analysis.


In [ ]:
# List of search terms, further terms can be added for analysis in this list
search_list = ['NHS','hiring','UK','US', 'AI']

# Initialize a list to store the results
tweets_with_search_results = []

# Iterate over each tweet in the cleaned tweets list
for tweet in tweets_clean:
    # Create a dictionary to store the search results for this tweet
    search_results = {term: False for term in search_list}

    # Check each search term in the tweet (case-insensitive)
    for term in search_list:
        if term.lower() in tweet.lower():
            search_results[term] = True

    # Append the tweet and its search results to the results list
    tweets_with_search_results.append({
        'tweet': tweet,
        'search_results': search_results
    })

# Initialize a dictionary to store the count of each search term
search_term_counts = {term: 0 for term in search_list}

# Iterate over each tweet's search results and count the occurrences of each search term
for item in tweets_with_search_results:
    for term, found in item['search_results'].items():
        if found:
            search_term_counts[term] += 1

# Print the count of tweets containing each search term
for term, count in search_term_counts.items():
    print(f"Number of tweets containing '{term}': {count}")